### 1. Разделение видео на интервалы и извлечение кадров

* Функция split_video_into_frames:
* Открывает видеофайл с помощью OpenCV.
* Получает количество кадров в секунду (fps).
* Вычисляет количество кадров в каждом интервале.
* Читает кадры из видео и собирает их в списки фиксированной длины.
* Возвращает список интервалов (где каждый интервал — это набор кадров) и количество кадров в секунду.

In [1]:
import cv2
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

# from pytorchvideo.models.hub import mvit_base_16x4


def split_video_into_frames(video_path, interval_sec):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval_frames = int(fps * interval_sec)
    frames = []
    intervals = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        if len(frames) == interval_frames:
            intervals.append(np.array(frames))
            frames = []

    cap.release()
    return intervals, fps

/Users/bogdanroshchupkin/anaconda3/envs/hackaton/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/bogdanroshchupkin/anaconda3/envs/hackaton/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: <D9493EF5-8DAB-3A5D-85D5-684F04544B84> /Users/bogdanroshchupkin/anaconda3/envs/hackaton/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <3F9923D2-81A5-3EC8-9739-EC0C1C816132> /Users/bogdanroshchupkin/anaconda3/envs/hackaton/lib/python3.10/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


ModuleNotFoundError: No module named 'pytorchvideo'

In [2]:
device = "cpu"

# Pick a pretrained model and load the pretrained weights
model_name = "slowfast_r50"
model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)

# Set to eval mode and move to desired device
model = model.to(device)
model = model.eval()

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /Users/bogdanroshchupkin/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOWFAST_8x8_R50.pyth" to /Users/bogdanroshchupkin/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R50.pyth
100%|████████████████████████████████████████| 264M/264M [00:24<00:00, 11.1MB/s]


In [3]:
model

Net(
  (blocks): ModuleList(
    (0): MultiPathWayWithFuse(
      (multipathway_blocks): ModuleList(
        (0): ResNetBasicStem(
          (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
          (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
        (1): ResNetBasicStem(
          (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
          (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
      )
      (multipathway_fusion): FuseFastToSlow(
        (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), st

### 2. Разметка интервалов
* Разметка интервалов:
* Создает список меток для каждого интервала. В этом примере все метки устанавливаются как 'no_violation', но в реальной задаче их нужно заменить на соответствующие метки нарушений.

In [ ]:
labels = ['no_violation'] * len(intervals)  # Заменить на реальные метки


### 3. Создание датасета и загрузчика данных
* Класс VideoDataset:

* Наследуется от torch.utils.data.Dataset.
* Инициализируется интервалами и метками.
* Метод __len__ возвращает количество интервалов.
* Метод __getitem__ возвращает интервал и соответствующую метку по индексу. При необходимости применяет трансформации.
** Трансформации:

* Нормализация пиксельных значений до диапазона [0, 1].
* Изменение размера и центрирование для создания одинаковых входных данных.
** DataLoader:

* Создает загрузчик данных для обучения модели, который обеспечивает удобное и эффективное чтение данных пакетами (batch_size=4).

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, intervals, labels, transform=None):
        self.intervals = intervals
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.intervals)
    
    def __getitem__(self, idx):
        interval = self.intervals[idx]
        label = self.labels[idx]
        if self.transform:
            interval = self.transform(interval)
        return interval, label

# Трансформация для модели SlowFast
transform = transforms.Compose([
    transforms.Lambda(lambda x: x / 255.0),  # Нормализация
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

dataset = VideoDataset(intervals, labels, transform=transform)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


### 4. Загрузка предобученной модели SlowFast
* Загружает предобученную модель SlowFast (ResNet50) из PyTorchVideo.
* Заменяет последний слой модели (model.blocks[-1].proj) для адаптации под количество классов в вашей задаче.
* Переносит модель на GPU для ускорения вычислений.

In [ ]:
# Загрузка предобученной модели
model = slowfast_r50(pretrained=True)

# Замена последнего слоя для нашей задачи
model.blocks[-1].proj = nn.Linear(model.blocks[-1].proj.in_features, num_classes)
model = model.cuda()


### 5. Обучение модели

* Оптимизатор и функция потерь:

* Использует Adam для оптимизации параметров модели.
* Функция потерь — кросс-энтропийная (CrossEntropyLoss), подходящая для задачи классификации.
* Функция train_model:

* Переводит модель в режим обучения.
* Выполняет обучение в течение заданного количества эпох.
* Для каждого пакета данных обнуляет градиенты, выполняет прямой проход, вычисляет потерю, выполняет обратный проход и обновляет параметры модели.
* Суммирует и выводит потери на каждой эпохе.

In [ ]:
import torch.optim as optim

learning_rate = 0.001
num_epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

train_model(model, train_loader, criterion, optimizer, num_epochs)


### 6. Оценка модели

* Функция evaluate_model:
* Переводит модель в режим оценки.
* Отключает вычисление градиентов для экономии памяти и ускорения.
* Выполняет предсказания и сравнивает их с истинными метками, вычисляя точность.

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

# Пример использования для тестового набора
test_loader = DataLoader(dataset, batch_size=4, shuffle=False)  # Заменить на реальный тестовый набор
evaluate_model(model, test_loader)


### 7. Генерация статистики
* Функция generate_statistics:

* Создает словарь статистики, где ключами являются типы нарушений, а значениями — списки временных интервалов.
* Для каждого предсказания вычисляет начало и конец интервала и добавляет их в соответствующий список.
* Генерация и вывод статистики:

* Выполняет предсказания для тестового набора данных.
* Создает и выводит статистику по каждому типу нарушения.

In [ ]:
def generate_statistics(predictions, interval_sec):
    stats = {}
    for i, prediction in enumerate(predictions):
        violation_type = prediction.item()
        if violation_type not in stats:
            stats[violation_type] = []
        start_time = i * interval_sec
        end_time = (i + 1) * interval_sec
        stats[violation_type].append((start_time, end_time))
    return stats

# Предсказания на тестовом наборе данных
model.eval()
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted)

# Создание статистики
interval_sec = 5  # Длина интервала в секундах
statistics = generate_statistics(all_predictions, interval_sec)

# Вывод статистики
for violation, intervals in statistics.items():
    print(f"Violation type {violation}:")
    for interval in intervals:
        print(f"  Start: {interval[0]} sec, End: {interval[1]} sec")


### Первый не рабочий поэтому второй и рабочий вариант

Load the model:

In [ ]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

Import remaining functions:

In [ ]:
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
) 

In [ ]:
Set the model to eval mode and move to desired device.

In [ ]:
# Set to GPU or CPU
device = "cpu"
model = model.eval()
model = model.to(device)


Download the id to label mapping for the Kinetics 400 dataset on which the torch hub models were trained. This will be used to get the category label names from the predicted class ids.

# Рабочий инференс!!!

In [13]:
import time
import torch
# Choose the `slowfast_r50` model 
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
) 

# Set to GPU or CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.eval()
model = model.to(device)

json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try:
    urllib.URLopener().retrieve(json_url, json_filename)
except:
    urllib.request.urlretrieve(json_url, json_filename)

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors. 
    """
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate) / frames_per_second

# Use the local video file for testing
# video_path = 'barbequing/0g-1i2ZSnwU_000124_000134.mp4'
video_path = 'dataset_attempt/train/barbequing/zXJEF2t7-ZM_000134_000144.mp4'
# 'ybSoSE_2gWg_000386_000396.mp4',
# 'ykTlbzf2Rds_000041_000051.mp4',
# 'YVL7IegDddA_000287_000297.mp4',
# 'yWUUxrJ6eqY_000330_000340.mp4',  
# 'Z-vvXMjXVZ4_000197_000207.mp4',
# 'ZshjCXHNAMM_000014_000024.mp4',
# 'zXJEF2t7-ZM_000134_000144.mp4'

# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0
end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class and load the video
video = EncodedVideo.from_path(video_path)

# # Load the desired clip
# video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# # Apply a transform to normalize the video input
# video_data = transform(video_data)

# # Move the inputs to the desired device
# inputs = video_data["video"]
# inputs = [i.to(device)[None, ...] for i in inputs]

# # Pass the input clip through the model
# start_time = time.time()
# # Pass the input clip through the model
# preds = model(inputs)
# end_time = time.time()

# # Вывод времени предсказания
# print(f"Prediction time: {end_time - start_time:.4f} seconds")

# # Get the predicted classes
# post_act = torch.nn.Softmax(dim=1)
# preds = post_act(preds)
# pred_classes = preds.topk(k=5).indices[0]

# # Map the predicted classes to the label names
# pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
# print("Top 5 predicted labels: %s" % ", ".join(pred_class_names))


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


Prediction time: 0.8746 seconds
Top 5 predicted labels: cooking chicken, cooking sausages, scrambling eggs, breading or breadcrumbing, barbequing


# Обучение модели

In [14]:
import time
import torch
import json
import urllib
from typing import Dict
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 1)  # Single output for binary classification
model = model.to(device)

# Загрузка классов Kinetics
json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try:
    urllib.URLopener().retrieve(json_url, json_filename)
except:
    urllib.request.urlretrieve(json_url, json_filename)

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Создание словаря для маппинга id на имена классов
kinetics_id_to_classname = {v: k for k, v in kinetics_classnames.items()}

# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# Длительность клипа
clip_duration = (num_frames * sampling_rate) / frames_per_second


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [23]:
import os
import glob

class VideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '*.mp4'))
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video = EncodedVideo.from_path(video_path)
        start_sec = 0
        end_sec = (num_frames * sampling_rate) / frames_per_second
        video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
        if self.transform:
            video_data = self.transform(video_data)
        video_tensor = video_data["video"]
        return video_tensor, torch.tensor(0)  # Assuming single class 'barbequing' with label 0

# Путь к вашим тренировочным видео
train_video_dir = 'dataset_attempt/train/barbequing'
train_dataset = VideoDataset(video_dir=train_video_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Проверка работы train_loader
def check_videos(video_dir):
    video_paths = glob.glob(os.path.join(video_dir, '*.mp4'))
    for video_path in video_paths:
        try:
            video = EncodedVideo.from_path(video_path)
            start_sec = 0
            end_sec = (num_frames * sampling_rate) / frames_per_second
            video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
            print(f"Successfully loaded video: {video_path}")
        except Exception as e:
            print(f"Failed to load video: {video_path}. Error: {e}")

# Путь к вашим тренировочным видео
train_video_dir = 'dataset_attempt/train/barbequing'
num_frames = 32
sampling_rate = 2
frames_per_second = 30

# Запуск проверки видеофайлов
check_videos(train_video_dir)


Successfully loaded video: dataset_attempt/train/barbequing/ykTlbzf2Rds_000041_000051.mp4
Successfully loaded video: dataset_attempt/train/barbequing/Z-vvXMjXVZ4_000197_000207.mp4
Successfully loaded video: dataset_attempt/train/barbequing/ZshjCXHNAMM_000014_000024.mp4
Successfully loaded video: dataset_attempt/train/barbequing/YVL7IegDddA_000287_000297.mp4
Successfully loaded video: dataset_attempt/train/barbequing/ybSoSE_2gWg_000386_000396.mp4
Successfully loaded video: dataset_attempt/train/barbequing/zXJEF2t7-ZM_000134_000144.mp4


In [24]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [25]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = [i.to(device) for i in inputs]
        labels = labels.float().to(device).view(-1, 1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs[0].size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/5, Loss: 0.3233
Epoch 2/5, Loss: 0.0767
Epoch 3/5, Loss: 0.0528
Epoch 4/5, Loss: 0.0570
Epoch 5/5, Loss: 0.0026


# Разбиение датасета на интервалы

In [ ]:
from moviepy.editor import *

clip = VideoFileClip("0000000_00000020240221082923_0001_IMP (1).mp4")

clip = clip.subclip(55, 60)

clip.write_videofile("gfg_intro.mp4",audio=False, fps=5)

In [35]:
import pandas as pd
import os

# Путь к Excel файлу
csv_file_path = 'Razmetka.xlsx'

# Путь к директории с видеофайлами
video_dir = 'train_hack'

# Загрузка данных из Excel файла
df = pd.read_excel(csv_file_path)

# Получение списка видеофайлов из столбца "video"
video_filenames_from_csv = df['video'].tolist()

# Вывод названий файлов из датасета
print("Названия файлов из датасета:")
for filename in video_filenames_from_csv:
    print(filename)

# Получение списка всех файлов в директории video_dir
video_filenames_in_dir = os.listdir(video_dir)

# Фильтрация только файлов с расширением .mp4
video_filenames_in_dir = [f for f in video_filenames_in_dir if f.endswith(('.mp4', '.MP4'))]

# Вывод названий файлов в директории
print("\nНазвания файлов в директории:")
for filename in video_filenames_in_dir:
    print(filename)

# Проверка наличия каждого файла из CSV в директории
missing_files = [f for f in video_filenames_from_csv if f not in video_filenames_in_dir]

if not missing_files:
    print("\nВсе названия файлов совпадают.")
else:
    print("\nСледующие файлы не найдены в директории:")
    for missing_file in missing_files:
        print(missing_file)


Названия файлов из датасета:
VID-20240301-WA0023.mp4
VID-20240301-WA0022.mp4
VID-20240301-WA0022.mp4
VID-20240227-WA0030.mp4
VID_20240313_155808.mp4
VID_20240313_155451.mp4
VID_20240313_155411.mp4
VID_20240313_155312.mp4
VID_20240313_155201.mp4
VID_20240313_155129.mp4
VID_20240313_155059.mp4
VID_20240313_155024.mp4
VID_20240311_155103.mp4
VID_20240311_154843.mp4
VID_20240311_154807.mp4
VID_20240311_154731.mp4
VID_20240311_115938.mp4
VID_20240311_115528.mp4
VID_20240311_115448.mp4
VID_20240305_130713.mp4
VID_20240305_130642.mp4
VID_20240305_130357.mp4
VID_20240305_130307.mp4
VID_20240304_095423.mp4
VID_20240304_095352.mp4
VID_20240304_095302.mp4
VID_20240304_095227.mp4
VID_20240304_095112.mp4
27e2fbc6-8579-4f48-852d-a42fd49522a2.MP4
2_5395803543229709828.mp4
2_5395803543229709828.mp4
2_5395803543229709728.mp4
0000000_00000020240221082923_0001_IMP.MP4
0000000_00000020240221082923_0001_IMP.MP4
0000000_00000020240221082923_0001_IMP.MP4
0000000_00000020240221084423_0002_IMP.MP4
0000000_0000

### Разбиение(тестирование)

In [69]:
%%time

import pandas as pd
import os
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from datetime import datetime, time

# Загрузка данных из Excel файла
csv_file_path = 'Razmetka.xlsx'
df = pd.read_excel(csv_file_path)

# Путь к директории с видеофайлами
video_dir = 'train_hack'

def time_to_seconds(time_str):
    print(f"Original time: {time_str}, type: {type(time_str)}")
    if isinstance(time_str, pd.Timestamp):
        # Преобразование Timestamp обратно в строку в формате '%M:%S'
        time_str = time_str.strftime('%M:%S')
    elif isinstance(time_str, time):
        # Преобразование времени обратно в строку в формате '%M:%S'
        time_str = time_str.strftime('%M:%S')
    else:
        # Преобразование строки в формат '%M:%S'
        time_str = str(time_str)

    print(f"Converted time: {time_str}")
    time_obj = datetime.strptime(time_str, '%M:%S')
    return time_obj.minute * 60 + time_obj.second

def extract_intervals(video_file, df):
    # Получение списка интервалов для данного видео
    intervals = df[df['video'] == video_file][['start', 'end', 'group']]
    print(intervals)

    if intervals.empty:
        print(f"No intervals found for video: {video_file}")
        return

    # Путь к видео файлу
    video_path = os.path.join(video_dir, video_file)

    # Проверка на наличие None в значениях "start" и "end"
    if intervals['start'].isnull().any() or intervals['end'].isnull().any():
        print(f"Видео {video_file} имеет None значения в 'start' или 'end'. Переносим в 'group_remaining'.")
        remaining_output_dir = "group_remaining"
        if not os.path.exists(remaining_output_dir):
            os.makedirs(remaining_output_dir)
        remaining_output_file = os.path.join(remaining_output_dir, video_file)
        os.rename(video_path, remaining_output_file)
        return

    # Получение общего списка всех интервалов
    all_intervals = []
    for _, row in intervals.iterrows():
        start = row['start']
        end = row['end']
        group = row['group']

        # Преобразование времени в секунды
        start_sec = time_to_seconds(start)
        end_sec = time_to_seconds(end)

        # Добавление интервала в список
        all_intervals.append((start_sec, end_sec, group))

    # Сортировка интервалов по времени начала
    all_intervals.sort()

    # Вырезка интервалов и сохранение видео
    for idx, (start_sec, end_sec, group) in enumerate(all_intervals):
        output_dir = f"group_{int(group)}"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        output_file = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}_{idx}.mp4")
        print(f"Extracting clip: start_sec={start_sec}, end_sec={end_sec}, output_file={output_file}")
        ffmpeg_extract_subclip(video_path, start_sec, end_sec, targetname=output_file)

    # Создание видео с оставшимися частями
    remaining_intervals = []
    prev_end = 0
    for start_sec, end_sec, group in all_intervals:
        if prev_end < start_sec:
            remaining_intervals.append((prev_end, start_sec))
        prev_end = end_sec

    # Добавление последнего оставшегося интервала
    video_cap = cv2.VideoCapture(video_path)
    total_duration = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT) / video_cap.get(cv2.CAP_PROP_FPS))
    if prev_end < total_duration:
        remaining_intervals.append((prev_end, total_duration))

    # Сохранение оставшихся интервалов
    remaining_output_dir = "group_remaining"
    if not os.path.exists(remaining_output_dir):
        os.makedirs(remaining_output_dir)

    for idx, (start_sec, end_sec) in enumerate(remaining_intervals):
        output_file = os.path.join(remaining_output_dir, f"{os.path.splitext(video_file)[0]}_remaining_{idx}.mp4")
        print(f"Extracting remaining clip: start_sec={start_sec}, end_sec={end_sec}, output_file={output_file}")
        ffmpeg_extract_subclip(video_path, start_sec, end_sec, targetname=output_file)

# Пример использования функции
extract_intervals('VID-20240301-WA0023.mp4', df)


      start       end  group
0  00:02:16  00:02:23    1.0
Original time: 00:02:16, type: <class 'datetime.time'>
Converted time: 02:16
Original time: 00:02:23, type: <class 'datetime.time'>
Converted time: 02:23
Extracting clip: start_sec=136, end_sec=143, output_file=group_1/VID-20240301-WA0023_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting remaining clip: start_sec=0, end_sec=136, output_file=group_remaining/VID-20240301-WA0023_remaining_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting remaining clip: start_sec=143, end_sec=327, output_file=group_remaining/VID-20240301-WA0023_remaining_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
CPU times: user 17.7 ms, sys: 10.4 ms, total: 28.1 ms
Wall time: 212 ms


### Код по нарезке видео

In [ ]:
import os
import subprocess
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def get_total_frames(video_path):
    video_info_cmd = [
        'ffprobe', '-v', 'error', '-select_streams', 'v:0', '-count_frames',
        '-show_entries', 'stream=nb_read_frames', '-of', 'default=nokey=1:noprint_wrappers=1', video_path
    ]
    result = subprocess.run(video_info_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return int(result.stdout.strip())

def split_video(video_path, max_frames):
    total_frames = get_total_frames(video_path)
    
    # Get FPS
    fps_cmd = [
        'ffprobe', '-v', '0', '-select_streams', 'v:0', '-show_entries', 'stream=r_frame_rate',
        '-of', 'csv=p=0', video_path
    ]
    result = subprocess.run(fps_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    fps = eval(result.stdout.strip())
    
    # Calculate duration per chunk
    duration_per_chunk = max_frames / fps
    total_duration = total_frames / fps
    
    video_basename = os.path.basename(video_path)
    video_dir = os.path.dirname(video_path)
    output_dir = os.path.join(video_dir, "split_videos")
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    start = 0
    chunk_idx = 0
    
    while start < total_duration:
        end = min(start + duration_per_chunk, total_duration)
        output_file = os.path.join(output_dir, f"{os.path.splitext(video_basename)[0]}_{chunk_idx}.mp4")
        ffmpeg_extract_subclip(video_path, start, end, targetname=output_file)
        start = end
        chunk_idx += 1
    
    os.remove(video_path)

def process_directory(directory, max_frames):
    for video_file in os.listdir(directory):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(directory, video_file)
            split_video(video_path, max_frames)

# Пример использования функции
directory_path = 'train_hack'
max_frames = 300  # Замените на нужное количество кадров
process_directory(directory_path, max_frames)


### Функция подсчета FPS

In [119]:
import os
import subprocess
from fractions import Fraction

def get_fps(video_path):
    """Получает FPS видеофайла."""
    command = [
        'ffprobe', '-v', '0', '-select_streams', 'v:0', '-show_entries', 'stream=r_frame_rate',
        '-of', 'csv=p=0', video_path
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    fps_str = result.stdout.strip()
    fps = float(Fraction(fps_str))
    return fps

def iterate_and_print_fps(directory):
    """Проходит по всем видеофайлам в указанной директории и выводит их FPS."""
    for video_file in os.listdir(directory):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(directory, video_file)
            fps = get_fps(video_path)
            print(f"Video: {video_file}, FPS: {fps}")

# Пример использования функции
directory_path = 'group_remaining'
iterate_and_print_fps(directory_path)


Video: VID-20240301-WA0022_remaining_0.mp4, FPS: 29.97002997002997
Video: 0000000_00000020240221084423_0002_IMP_remaining_1.mp4, FPS: 29.97002997002997
Video: VID_20240304_095352_remaining_0.mp4, FPS: 60.0
Video: VID_20240311_154807.mp4, FPS: 60.0
Video: 0000000_00000020240221084423_0002_IMP_remaining_0.mp4, FPS: 29.97002997002997
Video: VID-20240301-WA0022_remaining_1.mp4, FPS: 29.97002997002997
Video: 0000000_00000020240221084423_0002_IMP_remaining_2.mp4, FPS: 29.97002997002997
Video: 2_5395803543229709261_remaining_0.mp4, FPS: 29.97002997002997
Video: VID_20240311_115938.mp4, FPS: 60.0
Video: 2_5395803543229709261_remaining_1.mp4, FPS: 29.97002997002997
Video: VID-20240301-WA0022_remaining_2.mp4, FPS: 29.97002997002997
Video: 0000000_00000020240221085923_0003_IMP_remaining_3.mp4, FPS: 29.97002997002997
Video: VID_20240313_155201.mp4, FPS: 60.0
Video: 0000000_00000020240221085923_0003_IMP_remaining_2.mp4, FPS: 29.97002997002997
Video: 0000000_00000020240221085923_0003_IMP_remaining_0

### Конвертация всех видеофайлов в самый популярный FPS

In [120]:
import os
import subprocess
from collections import Counter

def get_fps(video_path):
    """Получает FPS видеофайла."""
    command = [
        'ffprobe', '-v', '0', '-select_streams', 'v:0', '-show_entries', 'stream=r_frame_rate',
        '-of', 'csv=p=0', video_path
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    fps = eval(result.stdout.strip())
    return fps

def find_most_common_fps(directory):
    """Находит самое популярное значение FPS в указанной директории."""
    fps_list = []
    for video_file in os.listdir(directory):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(directory, video_file)
            fps = get_fps(video_path)
            fps_list.append(fps)
    
    fps_counter = Counter(fps_list)
    most_common_fps = fps_counter.most_common(1)[0][0]
    return most_common_fps

def convert_to_fixed_fps(video_path, output_path, target_fps):
    """Конвертирует видео в заданное значение FPS."""
    command = [
        'ffmpeg', '-i', video_path, '-r', str(target_fps), output_path
    ]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

def process_directory(directory, target_fps):
    """Проходит по всем видеофайлам в указанной директории и конвертирует их в заданное значение FPS."""
    output_dir = os.path.join(directory, "fixed_fps_videos")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for video_file in os.listdir(directory):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(directory, video_file)
            output_path = os.path.join(output_dir, video_file)
            convert_to_fixed_fps(video_path, output_path, target_fps)
            print(f"Converted {video_file} to {target_fps} FPS")

# Пример использования функций
directory_path = 'group_remaining'
most_common_fps = find_most_common_fps(directory_path)
process_directory(directory_path, most_common_fps)


Converted VID-20240301-WA0022_remaining_0.mp4 to 29.97002997002997 FPS
Converted 0000000_00000020240221084423_0002_IMP_remaining_1.mp4 to 29.97002997002997 FPS
Converted VID_20240304_095352_remaining_0.mp4 to 29.97002997002997 FPS
Converted VID_20240311_154807.mp4 to 29.97002997002997 FPS
Converted 0000000_00000020240221084423_0002_IMP_remaining_0.mp4 to 29.97002997002997 FPS
Converted VID-20240301-WA0022_remaining_1.mp4 to 29.97002997002997 FPS
Converted 0000000_00000020240221084423_0002_IMP_remaining_2.mp4 to 29.97002997002997 FPS
Converted 2_5395803543229709261_remaining_0.mp4 to 29.97002997002997 FPS
Converted VID_20240311_115938.mp4 to 29.97002997002997 FPS
Converted 2_5395803543229709261_remaining_1.mp4 to 29.97002997002997 FPS
Converted VID-20240301-WA0022_remaining_2.mp4 to 29.97002997002997 FPS
Converted 0000000_00000020240221085923_0003_IMP_remaining_3.mp4 to 29.97002997002997 FPS
Converted VID_20240313_155201.mp4 to 29.97002997002997 FPS
Converted 0000000_0000002024022108592

## Аугментации

In [78]:
import os
import cv2
import torch
import numpy as np
import kornia.augmentation as K
from moviepy.editor import VideoFileClip, concatenate_videoclips
from moviepy.video.fx.all import speedx
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from torchvision.transforms import Compose, Lambda

# Определение аугментаций для тензоров
video_augmentations = torch.nn.Sequential(
    K.RandomHorizontalFlip(p=0.5),
    K.RandomRotation(degrees=15.0),
    K.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)
)

# Дополнительные аугментации для тензоров
def add_noise(video_tensor):
    noise = torch.randn_like(video_tensor) * 0.1
    return video_tensor + noise

def temporal_shift(video_tensor, shift):
    return torch.roll(video_tensor, shifts=shift, dims=0)

def frame_drop(video_tensor, drop_prob):
    mask = torch.rand(video_tensor.size(0)) > drop_prob
    return video_tensor[mask]

def frame_duplicate(video_tensor, duplicate_prob):
    mask = torch.rand(video_tensor.size(0)) < duplicate_prob
    duplicated_frames = video_tensor[mask]
    return torch.cat([video_tensor, duplicated_frames], dim=0)

# Дополнительные аугментации для VideoFileClip
def adjust_speed(video_clip, factor):
    return speedx(video_clip, factor)

# Функция для аугментации видео
def augment_video(video_path, output_path, tensor_augmentations, clip_augmentations):
    # Загрузка видео
    print("Loading video...")
    video_clip = VideoFileClip(video_path)
    fps = video_clip.fps
    frames = []

    # Разделение видео на кадры и преобразование в тензор
    print("Converting video to tensor...")
    for frame in video_clip.iter_frames():
        frame_tensor = torch.from_numpy(frame).permute(2, 0, 1).float() / 255.0
        frames.append(frame_tensor)

    video_tensor = torch.stack(frames)  # (T, C, H, W)

    # Применение временных аугментаций
    print("Applying temporal augmentations...")
    for aug in tensor_augmentations:
        if aug in [temporal_shift, frame_drop, frame_duplicate]:
            video_tensor = aug(video_tensor)

    # Применение пространственных и цветовых аугментаций к каждому кадру отдельно
    print("Applying frame augmentations...")
    augmented_frames = []
    for frame_tensor in video_tensor:  # (C, H, W)
        frame_tensor = frame_tensor.unsqueeze(0)  # (1, C, H, W)
        for aug in tensor_augmentations:
            if aug not in [temporal_shift, frame_drop, frame_duplicate]:
                try:
                    frame_tensor = aug(frame_tensor)
                except Exception as e:
                    print(f"Augmentation failed for frame with shape {frame_tensor.shape}: {e}")
        frame_tensor = torch.clamp(frame_tensor, 0, 1)
        augmented_frames.append(frame_tensor.squeeze(0))

    augmented_tensor = torch.stack(augmented_frames)  # (T, C, H, W)

    # Преобразование тензора обратно в кадры
    print("Converting tensor back to frames...")
    augmented_frames = augmented_tensor.permute(0, 2, 3, 1).cpu().numpy()
    augmented_frames = (augmented_frames * 255).astype(np.uint8)

    # Сохранение аугментированных кадров в виде видео
    print("Saving augmented frames to video...")
    height, width, _ = augmented_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    intermediate_path = 'intermediate_video.mp4'
    out = cv2.VideoWriter(intermediate_path, fourcc, fps, (width, height))

    for frame in augmented_frames:
        out.write(frame)

    out.release()

    # Применение аугментаций к VideoFileClip
    print("Applying clip augmentations...")
    augmented_clip = VideoFileClip(intermediate_path)
    for aug in clip_augmentations:
        augmented_clip = aug(augmented_clip)

    augmented_clip.write_videofile(output_path, fps=fps, codec="libx264", audio_codec="aac")
    os.remove(intermediate_path)

# Пример использования функции
video_path = 'group_1/0000000_00000020240221082923_0001_IMP_2.mp4'
output_path = 'augmented_video.mp4'
tensor_augmentations = [add_noise, lambda x: temporal_shift(x, 2), lambda x: frame_drop(x, 0.1), lambda x: frame_duplicate(x, 0.05), video_augmentations]
clip_augmentations = [lambda x: adjust_speed(x, 1.5)]
augment_video(video_path, output_path, tensor_augmentations, clip_augmentations)



Loading video...
Converting video to tensor...
Applying temporal augmentations...
Applying frame augmentations...
Augmentation failed for frame with shape torch.Size([0, 3, 1080, 1920]): Expected non-zero, positive output size. Got [0, 3, 1080, 1920]
Augmentation failed for frame with shape torch.Size([0, 3, 1080, 1920]): Expected non-zero, positive output size. Got [0, 3, 1080, 1920]
Augmentation failed for frame with shape torch.Size([0, 3, 1080, 1920]): Expected non-zero, positive output size. Got [0, 3, 1080, 1920]
Augmentation failed for frame with shape torch.Size([0, 3, 1080, 1920]): Expected non-zero, positive output size. Got [0, 3, 1080, 1920]
Augmentation failed for frame with shape torch.Size([0, 3, 1080, 1920]): Expected non-zero, positive output size. Got [0, 3, 1080, 1920]
Augmentation failed for frame with shape torch.Size([0, 3, 1080, 1920]): Expected non-zero, positive output size. Got [0, 3, 1080, 1920]
Augmentation failed for frame with shape torch.Size([0, 3, 1080,

RuntimeError: stack expects each tensor to be equal size, but got [3, 1080, 1920] at entry 0 and [0, 3, 1080, 1920] at entry 4

### Подготовка датасета

In [122]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import speedx

# Функция для дополнения или обрезки клипа до фиксированной длины
def pad_or_crop_clip(video_clip, target_duration, output_path, idx):
    fps = video_clip.fps
    current_duration = video_clip.duration

    if current_duration < target_duration:
        # Если видео короче целевой длительности, заполним недостающие кадры последним кадром (freeze frame)
        speed_factor = current_duration / target_duration
        augmented_clip = speedx(video_clip, factor=speed_factor)
        augmented_clip.write_videofile(f"{output_path}_{idx}.mp4", fps=fps, codec="libx264", audio_codec="aac")
    else:
        start_time = 0
        clip_idx = 0
        while start_time < current_duration:
            end_time = start_time + target_duration
            if end_time > current_duration:
                remaining_duration = current_duration - start_time
                if remaining_duration < 3:
                    break
                else:
                    end_time = current_duration

            subclip = video_clip.subclip(start_time, end_time)
            subclip_frames = list(subclip.iter_frames())
            height, width, _ = subclip_frames[0].shape
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            subclip_out = cv2.VideoWriter(f"{output_path}_{idx}_{clip_idx}.mp4", fourcc, fps, (width, height))
            
            for frame in subclip_frames:
                frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                subclip_out.write(frame_bgr)

            if end_time > current_duration or (end_time == current_duration and current_duration < target_duration):
                # Добавление замороженных кадров
                last_frame_bgr = cv2.cvtColor(subclip_frames[-1], cv2.COLOR_RGB2BGR)
                num_padding_frames = int((target_duration - (end_time - start_time)) * fps)
                for _ in range(num_padding_frames):
                    subclip_out.write(last_frame_bgr)

            subclip_out.release()
            start_time += target_duration
            clip_idx += 1

# Функция для подготовки датасета
def prepare_dataset(video_dir, output_dir, target_duration):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for idx, video_file in enumerate(os.listdir(video_dir)):
        if video_file.endswith(('.mp4', '.MP4')):
            video_path = os.path.join(video_dir, video_file)
            output_path = os.path.join(output_dir, os.path.splitext(video_file)[0])
            
            if os.path.exists(video_path):
                video_clip = VideoFileClip(video_path)
                # Применение дополнения или обрезки
                pad_or_crop_clip(video_clip, target_duration, output_path, idx)
            else:
                print(f"Видео {video_path} не найдено.")

# Пример использования функции
video_dir = 'group_remaining/fixed_fps_videos'
output_dir = 'output_directory/group_remaining'
target_duration = 5  # Целевая продолжительность в секундах
prepare_dataset(video_dir, output_dir, target_duration)


## Балансировка классов

In [124]:
import os
import shutil
import random
from math import ceil

def count_videos(directory):
    """Подсчитывает количество видеофайлов в указанной директории."""
    return len([file for file in os.listdir(directory) if file.endswith('.mp4')])

def copy_videos(src_dir, dst_dir, num_videos):
    """Копирует указанное количество случайных видеофайлов из src_dir в dst_dir."""
    files = [file for file in os.listdir(src_dir) if file.endswith('.mp4')]
    random.shuffle(files)
    for i in range(min(num_videos, len(files))):
        shutil.copy(os.path.join(src_dir, files[i]), os.path.join(dst_dir, files[i]))

def calculate_total_duration(directory):
    """Вычисляет общую длительность видеофайлов в указанной директории."""
    total_duration = 0
    for file in os.listdir(directory):
        if file.endswith('.mp4'):
            total_duration += 5  # Все видео по 5 секунд
    return total_duration / 60  # Переводим в минуты

def balance_dataset(output_directory, balanced_output_dir, group_1, group_3, group_remaining):
    if not os.path.exists(balanced_output_dir):
        os.makedirs(balanced_output_dir)

    # Подсчитываем количество видео в каждой группе
    group_1_count = count_videos(os.path.join(output_directory, group_1))
    group_3_count = count_videos(os.path.join(output_directory, group_3))
    
    # Общее количество нарушений
    total_violations = group_1_count + group_3_count

    # Количество видео, которое должно быть в группе "group_remaining" для 60% от общего количества
    remaining_count = ceil(total_violations * 1.5)  # 60% должно быть больше на 20% по продолжительности

    # Копируем видеофайлы в новую директорию для сбалансированного датасета
    os.makedirs(os.path.join(balanced_output_dir, group_1), exist_ok=True)
    os.makedirs(os.path.join(balanced_output_dir, group_3), exist_ok=True)
    os.makedirs(os.path.join(balanced_output_dir, group_remaining), exist_ok=True)

    copy_videos(os.path.join(output_directory, group_1), os.path.join(balanced_output_dir, group_1), group_1_count)
    copy_videos(os.path.join(output_directory, group_3), os.path.join(balanced_output_dir, group_3), group_3_count)
    copy_videos(os.path.join(output_directory, group_remaining), os.path.join(balanced_output_dir, group_remaining), remaining_count)

    # Вычисляем общую длительность видео в каждой группе
    total_duration_group_1 = calculate_total_duration(os.path.join(balanced_output_dir, group_1))
    total_duration_group_3 = calculate_total_duration(os.path.join(balanced_output_dir, group_3))
    total_duration_group_remaining = calculate_total_duration(os.path.join(balanced_output_dir, group_remaining))

    # Выводим статистику
    print(f"Total duration for {group_1}: {total_duration_group_1:.2f} minutes")
    print(f"Total duration for {group_3}: {total_duration_group_3:.2f} minutes")
    print(f"Total duration for {group_remaining}: {total_duration_group_remaining:.2f} minutes")

# Пример использования функции
output_directory = 'output_directory'
balanced_output_dir = 'balanced_output_directory'
group_1 = 'group_1'
group_3 = 'group_3'
group_remaining = 'group_remaining'

balance_dataset(output_directory, balanced_output_dir, group_1, group_3, group_remaining)


Total duration for group_1: 2.42 minutes
Total duration for group_3: 5.92 minutes
Total duration for group_remaining: 12.50 minutes


In [6]:
import os
import shutil
import random
from math import ceil

def count_videos(directory):
    """Подсчитывает количество видеофайлов в указанной директории."""
    if not os.path.exists(directory):
        return 0
    return len([file for file in os.listdir(directory) if file.endswith(('.mp4', '.MP4'))])

def copy_videos(src_dir, dst_dir, num_videos, exclude_files=[]):
    """Копирует указанное количество случайных видеофайлов из src_dir в dst_dir."""
    if not os.path.exists(src_dir):
        return
    files = [file for file in os.listdir(src_dir) if file.endswith(('.mp4', '.MP4')) and file not in exclude_files]
    random.shuffle(files)
    for i in range(min(num_videos, len(files))):
        shutil.copy(os.path.join(src_dir, files[i]), os.path.join(dst_dir, files[i]))

def calculate_total_duration(directory):
    """Вычисляет общую длительность видеофайлов в указанной директории."""
    if not os.path.exists(directory):
        return 0
    total_duration = 0
    for file in os.listdir(directory):
        if file.endswith(('.mp4', '.MP4')):
            total_duration += 5  # Все видео по 5 секунд
    return total_duration / 60  # Переводим в минуты

def copy_specific_videos(src_dir, dst_dir, specific_files):
    """Копирует указанные видеофайлы из src_dir в dst_dir."""
    if not os.path.exists(src_dir):
        return
    for file in specific_files:
        shutil.copy(os.path.join(src_dir, file), os.path.join(dst_dir, file))

def prepare_train_valid_splits(output_directory, balanced_output_dir, group_1, group_3, group_remaining):
    train_dir = os.path.join(balanced_output_dir, 'train')
    valid_dir = os.path.join(balanced_output_dir, 'valid')

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(valid_dir):
        os.makedirs(valid_dir)

    # Создание подкаталогов
    for group in [group_1, group_3, group_remaining]:
        os.makedirs(os.path.join(train_dir, group), exist_ok=True)
        os.makedirs(os.path.join(valid_dir, group), exist_ok=True)

    # Определение видео для валидационного набора
    valid_group_1_files = ["VID-20240301-WA0022_1_13.mp4", "VID-20240301-WA0023_0_8_0.mp4", "2_5395803543229709215_0_9.mp4"]
    valid_group_3_files = ["0000000_00000020240221082923_0001_IMP_0_2_0.mp4", "0000000_00000020240221082923_0001_IMP_1_3_0.mp4"]
    valid_group_remaining_files = ["0000000_00000020240221084423_0002_IMP_remaining_2_6_15.mp4"]

    # Копирование указанных видео в папки валидации
    copy_specific_videos(os.path.join(output_directory, group_1), os.path.join(valid_dir, group_1), valid_group_1_files)
    copy_specific_videos(os.path.join(output_directory, group_3), os.path.join(valid_dir, group_3), valid_group_3_files)
    copy_specific_videos(os.path.join(output_directory, group_remaining), os.path.join(valid_dir, group_remaining), valid_group_remaining_files)

    # Копирование оставшихся видео в обучающий набор
    copy_videos(os.path.join(output_directory, group_1), os.path.join(train_dir, group_1), count_videos(os.path.join(output_directory, group_1)) - len(valid_group_1_files), valid_group_1_files)
    copy_videos(os.path.join(output_directory, group_3), os.path.join(train_dir, group_3), count_videos(os.path.join(output_directory, group_3)) - len(valid_group_3_files), valid_group_3_files)

    # Расчет количества видео для группы remaining (60% от total_violations)
    group_1_count = count_videos(os.path.join(train_dir, group_1))
    group_3_count = count_videos(os.path.join(train_dir, group_3))
    total_violations = group_1_count + group_3_count
    remaining_count = ceil(total_violations * 1.3)

    # Копирование видео для группы remaining
    copy_videos(os.path.join(output_directory, group_remaining), os.path.join(train_dir, group_remaining), remaining_count, valid_group_remaining_files)

    # Вычисляем общую длительность видео в каждой группе
    total_duration_group_1_train = calculate_total_duration(os.path.join(train_dir, group_1))
    total_duration_group_3_train = calculate_total_duration(os.path.join(train_dir, group_3))
    total_duration_group_remaining_train = calculate_total_duration(os.path.join(train_dir, group_remaining))
    total_duration_group_1_valid = calculate_total_duration(os.path.join(valid_dir, group_1))
    total_duration_group_3_valid = calculate_total_duration(os.path.join(valid_dir, group_3))
    total_duration_group_remaining_valid = calculate_total_duration(os.path.join(valid_dir, group_remaining))

    # Выводим статистику
    print(f"Total duration for {group_1} (train): {total_duration_group_1_train:.2f} minutes")
    print(f"Total duration for {group_3} (train): {total_duration_group_3_train:.2f} minutes")
    print(f"Total duration for {group_remaining} (train): {total_duration_group_remaining_train:.2f} minutes")
    print(f"Total duration for {group_1} (valid): {total_duration_group_1_valid:.2f} minutes")
    print(f"Total duration for {group_3} (valid): {total_duration_group_3_valid:.2f} minutes")
    print(f"Total duration for {group_remaining} (valid): {total_duration_group_remaining_valid:.2f} minutes")

# Пример использования функции
output_directory = 'balanced_output_directory'  # Директория, в которой находятся папки "group_1", "group_3", "group_remaining"
balanced_output_dir = 'resnet_dataset'
group_1 = 'group_1'
group_3 = 'group_3'
group_remaining = 'group_remaining'

prepare_train_valid_splits(output_directory, balanced_output_dir, group_1, group_3, group_remaining)


Total duration for group_1 (train): 2.17 minutes
Total duration for group_3 (train): 5.75 minutes
Total duration for group_remaining (train): 10.33 minutes
Total duration for group_1 (valid): 0.25 minutes
Total duration for group_3 (valid): 0.17 minutes
Total duration for group_remaining (valid): 0.08 minutes


### Подобрка параметров

In [125]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import glob

def analyze_videos(video_dir):
    video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True) + \
                  glob.glob(os.path.join(video_dir, '**/*.MP4'), recursive=True)
    
    resolutions = []
    lengths = []
    fps_list = []

    for video_path in video_paths:
        try:
            video = VideoFileClip(video_path)
            fps = video.fps
            width, height = video.size
            length = video.duration
            
            resolutions.append((width, height))
            lengths.append(length)
            fps_list.append(fps)
            
            print(f"Video: {video_path}, Resolution: {width}x{height}, FPS: {fps}, Length: {length} sec")
        
        except Exception as e:
            print(f"Failed to analyze video: {video_path}. Error: {e}")
    
    if len(resolutions) > 0:
        avg_width = np.mean([res[0] for res in resolutions])
        avg_height = np.mean([res[1] for res in resolutions])
        avg_length = np.mean(lengths)
        avg_fps = np.mean(fps_list)

        print(f"Average Resolution: {int(avg_width)}x{int(avg_height)}")
        print(f"Average Length: {avg_length:.2f} sec")
        print(f"Average FPS: {avg_fps:.2f}")
    else:
        print("No valid video files found.")

# Путь к вашим видеофайлам
video_dir = 'balanced_output_directory'

# Запуск анализа видеофайлов
analyze_videos(video_dir)


Video: balanced_output_directory/group_3/2_5395803543229709215_3_1_11.mp4, Resolution: 1280x720, FPS: 29.97002997002997, Length: 5.01 sec
Video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_17.mp4, Resolution: 1920x1080, FPS: 29.97002997002997, Length: 5.01 sec
Video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_16.mp4, Resolution: 1920x1080, FPS: 29.97002997002997, Length: 5.01 sec
Video: balanced_output_directory/group_3/2_5395803543229709215_3_1_10.mp4, Resolution: 1280x720, FPS: 29.97002997002997, Length: 5.01 sec
Video: balanced_output_directory/group_3/2_5395803543229709215_3_1_12.mp4, Resolution: 1280x720, FPS: 29.97002997002997, Length: 5.01 sec
Video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_28.mp4, Resolution: 1920x1080, FPS: 29.97002997002997, Length: 5.01 sec
Video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_14.mp4, Resolution: 1920x1080, FPS: 29

KeyboardInterrupt: 

# Обучение итогового датасета

In [126]:
import os
import torch
import json
import urllib
from typing import Dict
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
import glob


In [127]:
# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 1)  # Single output for binary classification
model = model.to(device)


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [128]:
# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30  # Можно округлить до 30 для удобства
slowfast_alpha = 4


class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)


In [129]:
# Длительность клипа
clip_duration = (num_frames * sampling_rate) / frames_per_second

class VideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True) + \
                           glob.glob(os.path.join(video_dir, '**/*.MP4'), recursive=True)
        self.class_to_idx = {
            "group_1": 1,  # Нарушение 1
            "group_3": 1,  # Нарушение 3
            "group_remaining": 0  # Без нарушений
        }
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video = EncodedVideo.from_path(video_path)
        start_sec = 0
        end_sec = (num_frames * sampling_rate) / frames_per_second
        video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
        if self.transform:
            video_data = self.transform(video_data)
        video_tensor = video_data["video"]
        class_name = os.path.basename(os.path.dirname(video_path))
        label = self.class_to_idx[class_name]
        return video_tensor, torch.tensor(label, dtype=torch.float32)

# Путь к вашим тренировочным видео
train_video_dir = 'balanced_output_directory'
train_dataset = VideoDataset(video_dir=train_video_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


In [130]:
# Проверка работы train_loader
def check_videos(video_dir):
    video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True) + \
                  glob.glob(os.path.join(video_dir, '**/*.MP4'), recursive=True)
    for video_path in video_paths:
        try:
            video = EncodedVideo.from_path(video_path)
            start_sec = 0
            end_sec = (num_frames * sampling_rate) / frames_per_second
            video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
            print(f"Successfully loaded video: {video_path}")
        except Exception as e:
            print(f"Failed to load video: {video_path}. Error: {e}")

# Запуск проверки видеофайлов
check_videos(train_video_dir)


Successfully loaded video: balanced_output_directory/group_3/2_5395803543229709215_3_1_11.mp4
Successfully loaded video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_17.mp4
Successfully loaded video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_16.mp4
Successfully loaded video: balanced_output_directory/group_3/2_5395803543229709215_3_1_10.mp4
Successfully loaded video: balanced_output_directory/group_3/2_5395803543229709215_3_1_12.mp4
Successfully loaded video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_28.mp4
Successfully loaded video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_14.mp4
Successfully loaded video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_15.mp4
Successfully loaded video: balanced_output_directory/group_3/0000000_00000020240221085923_0003_IMP_0_0_29.mp4
Successfully loaded video: balanced_output_directory/group

In [131]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = [i.to(device) for i in inputs]
        labels = labels.to(device).view(-1, 1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs[0].size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/5, Loss: 0.6805
Epoch 2/5, Loss: 0.6419
Epoch 3/5, Loss: 0.6053
Epoch 4/5, Loss: 0.6077
Epoch 5/5, Loss: 0.5194


In [136]:
def predict(video_dir, model, transform, device, batch_size=2):
    dataset = VideoDataset(video_dir=video_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = [i.to(device) for i in inputs]
            outputs = model(inputs)
            preds = torch.sigmoid(outputs)
            predictions.extend(preds.cpu().numpy())
            print(predictions)
    
    return predictions

# Пример использования функции предсказаний
valid_video_dir = 'balanced_output_directory/group_remaining'
predictions = predict(valid_video_dir, model, transform, device)
print(predictions)

[array([0.00159615], dtype=float32), array([0.47293282], dtype=float32)]
[array([0.00159615], dtype=float32), array([0.47293282], dtype=float32), array([0.00303015], dtype=float32), array([0.65857804], dtype=float32)]
[array([0.00159615], dtype=float32), array([0.47293282], dtype=float32), array([0.00303015], dtype=float32), array([0.65857804], dtype=float32), array([0.43331683], dtype=float32), array([0.43871343], dtype=float32)]
[array([0.00159615], dtype=float32), array([0.47293282], dtype=float32), array([0.00303015], dtype=float32), array([0.65857804], dtype=float32), array([0.43331683], dtype=float32), array([0.43871343], dtype=float32), array([0.000272], dtype=float32), array([0.0358432], dtype=float32)]
[array([0.00159615], dtype=float32), array([0.47293282], dtype=float32), array([0.00303015], dtype=float32), array([0.65857804], dtype=float32), array([0.43331683], dtype=float32), array([0.43871343], dtype=float32), array([0.000272], dtype=float32), array([0.0358432], dtype=flo

KeyboardInterrupt: 

### Разбиение датасета для извлечение интервалов по работника

In [138]:
import pandas as pd
import os
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from datetime import datetime, time

# Загрузка данных из CSV файла
csv_file_path = 'Razmetka (1).xlsx'
df = pd.read_excel(csv_file_path)

# Путь к директории с видеофайлами
video_dir = 'train_hack'

# Путь к директории для сохранения нарезанных видеофайлов
output_dir = 'extracted_clips'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def time_to_seconds(time_str):
    print(f"Original time: {time_str}, type: {type(time_str)}")
    if isinstance(time_str, pd.Timestamp):
        # Преобразование Timestamp обратно в строку в формате '%M:%S'
        time_str = time_str.strftime('%M:%S')
    elif isinstance(time_str, time):
        # Преобразование времени обратно в строку в формате '%M:%S'
        time_str = time_str.strftime('%M:%S')
    else:
        # Преобразование строки в формат '%M:%S'
        time_str = str(time_str)

    print(f"Converted time: {time_str}")
    time_obj = datetime.strptime(time_str, '%M:%S')
    return time_obj.minute * 60 + time_obj.second

def extract_intervals(video_file, df):
    # Получение списка интервалов для данного видео
    intervals = df[df['video'] == video_file][['start', 'end', 'Group']]
    print(intervals)

    if intervals.empty:
        print(f"No intervals found for video: {video_file}")
        return

    # Путь к видео файлу
    video_path = os.path.join(video_dir, video_file)

    # Проверка на наличие None в значениях "start" и "end"
    if intervals['start'].isnull().any() or intervals['end'].isnull().any():
        print(f"Видео {video_file} имеет None значения в 'start' или 'end'. Пропускаем.")
        return

    # Получение общего списка всех интервалов
    all_intervals = []
    for _, row in intervals.iterrows():
        start = row['start']
        end = row['end']
        group = row['Group']

        # Преобразование времени в секунды
        start_sec = time_to_seconds(start)
        end_sec = time_to_seconds(end)

        # Добавление интервала в список
        all_intervals.append((start_sec, end_sec, group))

    # Вырезка интервалов и сохранение видео
    for idx, (start_sec, end_sec, group) in enumerate(all_intervals):
        output_file = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}_{idx}.mp4")
        print(f"Extracting clip: start_sec={start_sec}, end_sec={end_sec}, output_file={output_file}")
        ffmpeg_extract_subclip(video_path, start_sec, end_sec, targetname=output_file)

# Пример использования функции
video_files = df['video'].unique()
for video_file in video_files:
    extract_intervals(video_file, df)


      start       end  Group
0  00:00:00  00:00:16      2
Original time: 00:00:00, type: <class 'datetime.time'>
Converted time: 00:00
Original time: 00:00:16, type: <class 'datetime.time'>
Converted time: 00:16
Extracting clip: start_sec=0, end_sec=16, output_file=extracted_clips/VID_20240311_154731_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
      start       end  Group
1  00:00:03  00:00:04      2
Original time: 00:00:03, type: <class 'datetime.time'>
Converted time: 00:03
Original time: 00:00:04, type: <class 'datetime.time'>
Converted time: 00:04
Extracting clip: start_sec=3, end_sec=4, output_file=extracted_clips/VID_20240304_095423_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
      start       end  Group
2  00:00:21  00:00:22      2
Original time: 00:00:21, type: <class 'datetime.time'>
Converted time: 00:21
Original time: 00:00:22, type: <class 'datetime.time'>
Converted time: 00:22
Extracting clip: start_sec=21, end_

### Обучение на многоклассовую классификацию

In [14]:
import time
import torch
import json
import urllib
from typing import Dict
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
import os
import glob
import numpy as np


In [3]:
import torch

In [4]:
# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=False)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Three outputs for multi-class classification
model = model.to(device)

# Путь к контрольной точке
checkpoint_path = 'slowfast_epoch_5.pth'

# Загрузка контрольной точки
checkpoint = torch.load(checkpoint_path)

# Восстановление весов модели
model.load_state_dict(checkpoint['model_state_dict'])

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Восстановление состояния оптимизатора
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Восстановление других параметров, если необходимо
start_epoch = checkpoint['epoch']
loss = checkpoint['loss']

Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [4]:
# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)


NameError: name 'ApplyTransformToKey' is not defined

In [38]:
# Длительность клипа
clip_duration = (num_frames * sampling_rate) / frames_per_second

class VideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
        self.class_to_idx = {
            "group_1": 0,  # Нарушение 1
            "group_3": 1,  # Нарушение 3
            "group_remaining": 2  # Без нарушений
        }
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video = EncodedVideo.from_path(video_path)
        start_sec = 0
        end_sec = (num_frames * sampling_rate) / frames_per_second
        video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
        if self.transform:
            video_data = self.transform(video_data)
        video_tensor = video_data["video"]
        class_name = os.path.basename(os.path.dirname(video_path))
        label = self.class_to_idx[class_name]
        return video_tensor, torch.tensor(label, dtype=torch.long)

# Путь к вашим тренировочным видео
train_video_dir = 'resnet_dataset/train'
train_dataset = VideoDataset(video_dir=train_video_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Путь к вашим валидационным видео
valid_video_dir = 'resnet_dataset/valid'
valid_dataset = VideoDataset(video_dir=valid_video_dir, transform=transform)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True)

In [39]:
# Проверка работы train_loader
def check_videos(video_dir):
    video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
    for video_path in video_paths:
        try:
            video = EncodedVideo.from_path(video_path)
            start_sec = 0
            end_sec = (num_frames * sampling_rate) / frames_per_second
            video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
            print(f"Successfully loaded video: {video_path}")
        except Exception as e:
            print(f"Failed to load video: {video_path}. Error: {e}")

# Путь к вашим тренировочным видео
train_video_dir = 'resnet_dataset/valid'
num_frames = 32
sampling_rate = 2
frames_per_second = 30

# Запуск проверки видеофайлов
check_videos(valid_video_dir)


Successfully loaded video: resnet_dataset/valid/group_3/0000000_00000020240221082923_0001_IMP_1_3_0.mp4
Successfully loaded video: resnet_dataset/valid/group_3/0000000_00000020240221082923_0001_IMP_0_2_0.mp4
Successfully loaded video: resnet_dataset/valid/group_1/VID-20240301-WA0022_1_13.mp4
Successfully loaded video: resnet_dataset/valid/group_1/2_5395803543229709215_0_9.mp4
Successfully loaded video: resnet_dataset/valid/group_1/VID-20240301-WA0023_0_8_0.mp4
Successfully loaded video: resnet_dataset/valid/group_remaining/0000000_00000020240221084423_0002_IMP_remaining_2_6_15.mp4


In [40]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = [i.to(device) for i in inputs]
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs[0].size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # Валидация
    model.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs = [i.to(device) for i in val_inputs]
            val_labels = val_labels.to(device)
            
            outputs = model(val_inputs)
            loss = criterion(outputs, val_labels)
            
            val_running_loss += loss.item() * val_inputs[0].size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
    
    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
 
    # Сохранение весов модели и состояния оптимизатора
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }
    torch.save(checkpoint, f'slowfast_epoch_{epoch+1}.pth')


Epoch 1/10, Loss: 0.3646
Validation Loss: 1.0197, Validation Accuracy: 0.6667
Epoch 2/10, Loss: 0.3102
Validation Loss: 1.0597, Validation Accuracy: 0.6667
Epoch 3/10, Loss: 0.2817
Validation Loss: 1.5183, Validation Accuracy: 0.6667
Epoch 4/10, Loss: 0.3017
Validation Loss: 1.0106, Validation Accuracy: 0.6667
Epoch 5/10, Loss: 0.3050
Validation Loss: 1.2036, Validation Accuracy: 0.6667


KeyboardInterrupt: 

In [41]:
# Функция для предсказаний
def predict(video_dir, model, transform, device, batch_size=2):
    dataset = VideoDataset(video_dir=video_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = [i.to(device) for i in inputs]
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions

# Пример использования функции предсказаний
valid_video_dir = 'resnet_dataset/valid'
predictions = predict(valid_video_dir, model, transform, device)
print(predictions)


[2, 2, 0, 2, 0, 2]


In [ ]:
# Пример использования функции предсказаний
valid_video_dir = 'resnet_dataset/valid'
predicted_intervals = predict_intervals(valid_video_dir, model, transform, device)
print(predicted_intervals)


### Аугментации

In [83]:
import os
import cv2
import albumentations as A
import numpy as np
from tqdm import tqdm
import concurrent.futures
import random
import pandas as pd
import shutil

# Чтение файла разметки
file_path = 'Updated_Razmetka.xlsx'
df = pd.read_excel(file_path)

# Преобразование данных в словарь интервалов
intervals = {}
for index, row in df.iterrows():
    video_name = row['video']
    start_sec = row['start']
    end_sec = row['end']
    group = row['group']
    
    if video_name not in intervals:
        intervals[video_name] = []
    intervals[video_name].append((start_sec, end_sec, group))

# Функции для аугментации видео
def augment_video(video_path, output_path, augmentations):
    # Открытие видеофайла
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Инициализация записи нового видеофайла
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Применение аугментаций к кадру
        augmented = augmentations(image=frame)['image']
        
        # Запись аугментированного кадра в новый видеофайфайл
        out.write(augmented)
    
    # Закрытие файлов
    cap.release()
    out.release()

def get_augmentations():
    return [
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.8),
        A.GaussianBlur(blur_limit=(3, 7), p=0.8),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8),
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.8),
    ]

def create_augmentations():
    all_augmentations = get_augmentations()
    chosen_augmentations = random.sample(all_augmentations, 2)
    return A.Compose(chosen_augmentations)

def process_video(file_info):
    input_path, output_path, aug_output_path = file_info
    shutil.copy(input_path, output_path)
    augmentations = create_augmentations()
    augment_video(input_path, aug_output_path, augmentations)

def augment_videos_in_directory(input_dir, output_dir, intervals):
    new_intervals = []
    file_info_list = []

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.mp4'):  # Проверка расширения файла
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)
                aug_output_path = os.path.join(output_dir, "aug_" + relative_path)
                
                # Создание поддиректорий в выходной директории, если не существует
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                os.makedirs(os.path.dirname(aug_output_path), exist_ok=True)
                
                file_info_list.append((input_path, output_path, aug_output_path))
                
                if file in intervals:
                    for start_sec, end_sec, group in intervals[file]:
                        new_intervals.append({
                            'video': relative_path,
                            'start': start_sec,
                            'end': end_sec,
                            'group': group
                        })
                        new_intervals.append({
                            'video': "aug_" + relative_path,
                            'start': start_sec,
                            'end': end_sec,
                            'group': group
                        })

    # Обработка видеофайлов параллельно с отображением прогресса
    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(process_video, file_info_list), total=len(file_info_list)))

    # Создание нового файла разметки
    new_df = pd.DataFrame(new_intervals)
    new_df.to_excel('Updated_Razmetka_new.xlsx', index=False)

# Путь к директории с видеофайлами
input_directory = 'main_dataset'
output_directory = 'main_dataset_augmented'

augment_videos_in_directory(input_directory, output_directory, intervals)


100%|███████████████████████████████████████████| 68/68 [03:11<00:00,  2.81s/it]


# Деление данных

In [53]:
import os
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Загрузка данных из Excel
df = pd.read_excel("Razmetka.xlsx")

# Фильтрация строк с непустыми значениями интервалов
filtered_df = df.dropna(subset=['start', 'end', 'group'])

# Функция для преобразования времени из формата datetime.time в секунды
def time_to_seconds(time_obj):
    return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second

# Преобразование времени в секунды и создание новых интервалов
intervals = []
for _, row in filtered_df.iterrows():
    video = row['video']
    start = time_to_seconds(row['start'])
    end = time_to_seconds(row['end'])
    group = int(row['group'])
    
    # Основной интервал с нарушением
    intervals.append((video, start, end, group))
    
    # Добавление интервалов с меткой "0" (нет нарушений)
    if start - 10 > 0:
        intervals.append((video, start - 10, start, 0))
    intervals.append((video, end, end + 10, 0))

# Создание новых папок для обрезанных видео
os.makedirs('main_dataset/train/group_1', exist_ok=True)
os.makedirs('main_dataset/train/group_3', exist_ok=True)
os.makedirs('main_dataset/train/group_0', exist_ok=True)
os.makedirs('main_dataset/valid/group_1', exist_ok=True)
os.makedirs('main_dataset/valid/group_3', exist_ok=True)
os.makedirs('main_dataset/valid/group_0', exist_ok=True)

# Пример разделения на тренировочную и валидационную выборки
train_intervals = intervals[:int(len(intervals) * 0.8)]
valid_intervals = intervals[int(len(intervals) * 0.8):]

# Функция для обрезки видео и сохранения в соответствующую папку
def process_intervals(intervals, base_path):
    for video, start, end, group in intervals:
        if group in [1, 3]:
            input_path = os.path.join(f'group_{group}', video)
        else:
            # Предполагаем, что если нет нарушений, видео может быть в одной из этих папок
            input_path = os.path.join('group_1', video)
            if not os.path.exists(input_path):
                input_path = os.path.join('group_3', video)
        
        output_dir = os.path.join(base_path, f'group_{group}')
        output_path = os.path.join(output_dir, f'{os.path.splitext(video)[0]}_{start}_{end}.mp4')
        
        if os.path.exists(input_path):
            ffmpeg_extract_subclip(input_path, start, end, targetname=output_path)
        else:
            print(f"File {input_path} not found.")

# Обработка тренировочных и валидационных интервалов
process_intervals(train_intervals, 'main_dataset/train')
process_intervals(valid_intervals, 'main_dataset/valid')

# Создание нового DataFrame с обновленными интервалами
new_intervals = pd.DataFrame(intervals, columns=['video', 'start', 'end', 'group'])

# Сохранение нового DataFrame в Excel
new_intervals.to_excel('Updated_Razmetka.xlsx', index=False)


File group_1/VID-20240301-WA0023.mp4 not found.
File group_3/VID-20240301-WA0023.mp4 not found.
File group_3/VID-20240301-WA0023.mp4 not found.
File group_1/VID-20240301-WA0022.mp4 not found.
File group_3/VID-20240301-WA0022.mp4 not found.
File group_3/VID-20240301-WA0022.mp4 not found.
File group_1/VID-20240301-WA0022.mp4 not found.
File group_3/VID-20240301-WA0022.mp4 not found.
File group_3/VID-20240301-WA0022.mp4 not found.
File group_1/VID_20240304_095352.mp4 not found.
File group_3/VID_20240304_095352.mp4 not found.
File group_1/2_5395803543229709828.mp4 not found.
File group_3/2_5395803543229709828.mp4 not found.
File group_3/2_5395803543229709828.mp4 not found.
File group_1/2_5395803543229709828.mp4 not found.
File group_3/2_5395803543229709828.mp4 not found.
File group_3/2_5395803543229709828.mp4 not found.
File group_3/0000000_00000020240221082923_0001_IMP.MP4 not found.
File group_3/0000000_00000020240221082923_0001_IMP.MP4 not found.
File group_3/0000000_0000002024022108292

In [88]:
# import os
# import pandas as pd
# import random
# from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# # Загрузка данных из Excel
# df = pd.read_excel("Razmetka.xlsx")

# # Фильтрация строк с непустыми значениями интервалов
# filtered_df = df.dropna(subset=['start', 'end', 'group'])

# # Функция для преобразования времени из формата datetime.time в секунды
# def time_to_seconds(time_obj):
#     return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second

# # Преобразование времени в секунды и создание новых интервалов
# intervals = []
# for _, row in filtered_df.iterrows():
#     video = row['video']
#     start = time_to_seconds(row['start'])
#     end = time_to_seconds(row['end'])
#     group = int(row['group'])
    
#     # Основной интервал с нарушением
#     intervals.append((video, start, end, group))
    
#     # Добавление интервалов с меткой "0" (нет нарушений)
#     random_interval_before = random.randint(10, 20)
#     random_interval_after = random.randint(10, 20)
#     if start - random_interval_before > 0:
#         intervals.append((video, start - random_interval_before, start, 0))
#     intervals.append((video, end, end + random_interval_after, 0))

# # Создание новых папок для обрезанных видео
# os.makedirs('main_dataset/train/group_1', exist_ok=True)
# os.makedirs('main_dataset/train/group_3', exist_ok=True)
# os.makedirs('main_dataset/train/group_0', exist_ok=True)
# os.makedirs('main_dataset/valid/group_1', exist_ok=True)
# os.makedirs('main_dataset/valid/group_3', exist_ok=True)
# os.makedirs('main_dataset/valid/group_0', exist_ok=True)

# # Пример разделения на тренировочную и валидационную выборки
# train_intervals = intervals[:int(len(intervals) * 0.8)]
# valid_intervals = intervals[int(len(intervals) * 0.8):]

# # Функция для поиска видеофайла с любым расширением
# def find_video_file(video_name, search_dir):
#     possible_extensions = ['.mp4', '.MP4']
#     for ext in possible_extensions:
#         video_path = os.path.join(search_dir, video_name + ext)
#         if os.path.exists(video_path):
#             return video_path
#     return None

# # Функция для обрезки видео и сохранения в соответствующую папку
# def process_intervals(intervals, base_path):
#     for video, start, end, group in intervals:
#         video_name, _ = os.path.splitext(video)  # убираем расширение файла
#         input_path = find_video_file(video_name, 'train_hack')
        
#         output_dir = os.path.join(base_path, f'group_{group}')
#         output_path = os.path.join(output_dir, f'{video_name}_{start}_{end}.mp4')
        
#         if input_path:
#             ffmpeg_extract_subclip(input_path, start, end, targetname=output_path)
#         else:
#             print(f"File {video} not found in train_hack.")

# # Обработка тренировочных и валидационных интервалов 
# process_intervals(train_intervals, 'main_dataset/train')
# process_intervals(valid_intervals, 'main_dataset/valid')

# # Создание нового DataFrame с обновленными интервалами
# new_intervals = pd.DataFrame(intervals, columns=['video', 'start', 'end', 'group'])

# # Сохранение нового DataFrame в Excel
# new_intervals.to_excel('Updated_Razmetka.xlsx', index=False)


# Обучение модели

In [1]:
import time
import torch
import json
import urllib
from typing import Dict, List, Tuple
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
import os
import glob
import numpy as np


/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functi

In [2]:
# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Three outputs for multi-class classification
model = model.to(device)


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [3]:
# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)


In [24]:
import os
import glob
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorchvideo.data.encoded_video import EncodedVideo

# Чтение файла разметки
file_path = 'Updated_Razmetka.xlsx'
df = pd.read_excel(file_path)

# Преобразование данных в словарь интервалов
intervals = {}
for index, row in df.iterrows():
    video_name = row['video']
    start_sec = row['start']
    end_sec = row['end']
    group = row['group']
    
    if video_name not in intervals:
        intervals[video_name] = []
    intervals[video_name].append((start_sec, end_sec, group))

class SegmentedVideoDataset(Dataset):
    def __init__(self, video_dir, intervals: dict, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
        self.intervals = intervals
        self.class_to_idx = {
            "group_1": 0,  # Нарушение 1
            "group_3": 1,  # Нарушение 3
            "group_0": 2   # Без нарушений
        }
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video = EncodedVideo.from_path(video_path)
        
        # Разбиваем видео на интервалы
        video_segments = []
        labels = []
        video_name = os.path.basename(video_path)
        if video_name in self.intervals:
            for start_sec, end_sec, group in self.intervals[video_name]:
                video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
                if self.transform:
                    video_data = self.transform(video_data)
                video_tensor = video_data["video"]
                label = self.class_to_idx[group]
                video_segments.append(video_tensor)
                labels.append(torch.tensor(label, dtype=torch.long))
        
        return video_segments, labels

# Пример преобразования для данных
transform = None  # Замените на нужное преобразование

# Путь к вашим тренировочным видео
train_video_dir = 'main_dataset_no_aug/train'
train_dataset = SegmentedVideoDataset(video_dir=train_video_dir, intervals=intervals, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Путь к вашим валидационным видео
valid_video_dir = 'main_dataset_no_aug/valid'
valid_dataset = SegmentedVideoDataset(video_dir=valid_video_dir, intervals=intervals, transform=transform)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True)


# Обучение модели

In [1]:
import re
import os
import glob
import time
import json
import urllib
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from tqdm import tqdm
from typing import Dict, List, Tuple
from moviepy.editor import VideoFileClip

/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functi

In [2]:
# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели SlowFast
base_model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

# Получение выходных признаков
out_features = base_model.blocks[-1].proj.in_features
base_model.blocks[-1].proj = nn.Identity()  # Убираем последний слой

# Определение класса модели с двумя головами
class SlowFastWithRegression(nn.Module):
    def __init__(self, base_model, out_features, num_classes=3):
        super(SlowFastWithRegression, self).__init__()
        self.base_model = base_model
        self.classifier_head = nn.Linear(out_features, num_classes)
        self.regression_head = nn.Linear(out_features, 2)
    
    def forward(self, x):
        x = self.base_model(x)
        class_out = self.classifier_head(x)
        regression_out = self.regression_head(x)
        return class_out, regression_out

# Инициализация модели
model = SlowFastWithRegression(base_model, out_features, num_classes=3)
model = model.to(device)

# Чтение файла разметки
file_path = 'Updated_Razmetka.xlsx'
df = pd.read_excel(file_path)


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [3]:
from torchvision import transforms

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list


In [5]:
# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

transform = ApplyTransformToKey(
    key="video",
    transform=transforms.Compose(
        [
            UniformTemporalSubsample(num_frames),
            transforms.Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

In [31]:
from torchvision.io import read_video

class SegmentedVideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
        self.class_to_idx = {
            "group_1": 0,  # Нарушение 1
            "group_3": 1,  # Нарушение 3
            "group_0": 2   # Без нарушений
        }
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        print(f"Loading video: {video_path}")
        video = EncodedVideo.from_path(video_path)
        
        # Извлекаем метку класса из пути
        class_label_str = os.path.basename(os.path.dirname(video_path))
        class_label = self.class_to_idx[class_label_str]
        
        # Определяем продолжительность видео
        try:
            video_duration = VideoFileClip(video_path).duration
        except Exception as e:
            print(f"Error getting duration for video {video_path}: {e}")
            return [], ([], [])
        
        print(f"Video duration: {video_duration} seconds")
        
        # Интервал от 0 до конца видео
        start_sec = 0
        end_sec = video_duration
        
        # Получаем клип
        try:
            video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
            if self.transform:
                video_data = self.transform(video_data)
            
            video_tensor = video_data["video"]
            
            # Проверка на тип video_tensor и его преобразование
            if isinstance(video_tensor, list):
                # Приведение всех видео к одинаковому размеру кадров
                max_frames = max([vt.shape[1] for vt in video_tensor])
                video_tensor = [torch.nn.functional.pad(vt, (0, 0, 0, 0, 0, max_frames - vt.shape[1])) for vt in video_tensor]
                video_tensor = torch.stack(video_tensor)
                
            regress_label = torch.tensor([start_sec, end_sec], dtype=torch.float32)
            
            print(f"Video tensor shape: {video_tensor.shape}")
            
            return [video_tensor], ([torch.tensor(class_label, dtype=torch.long)], [regress_label])
        except Exception as e:
            print(f"Error processing video {video_path}: {e}")
            return [], ([], [])




# Путь к вашим тренировочным видео
train_video_dir = 'main_dataset_no_aug/train'
train_dataset = SegmentedVideoDataset(video_dir=train_video_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Путь к вашим валидационным видео
valid_video_dir = 'main_dataset_no_aug/valid'
valid_dataset = SegmentedVideoDataset(video_dir=valid_video_dir, transform=transform)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True)

In [32]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision.io import read_video

# # Функция для визуализации видеофреймов
# def show_video_frames(video_tensor, title=None):
#     pass
#     # if video_tensor is None or len(video_tensor) == 0:
#     #     print("No video frames to display.")
#     #     return
    
#     # # Переводим тензор в numpy и нормализуем для отображения
#     # video_np = video_tensor[0].permute(1, 2, 3, 0).numpy()
#     # video_np = (video_np * 255).astype(np.uint8)
    
#     # fig, axs = plt.subplots(1, len(video_np), figsize=(15, 5))
#     # if title:
#     #     fig.suptitle(title)
#     # for i, frame in enumerate(video_np):
#     #     axs[i].imshow(frame)
#     #     axs[i].axis('off')
#     # plt.show()

In [33]:
# Функция для проверки работы даталоадера
def check_dataloader(dataloader, num_batches=1):
    for i, (video_segments, (class_labels, regress_labels)) in enumerate(dataloader):
        if i >= num_batches:
            break
        print(f"Batch {i+1}")
        
        for j in range(len(video_segments)):
            video_segment = video_segments[j]
            class_label = class_labels[j]
            regress_label = regress_labels[j]
            
            print(f"Video Segment {j+1}")
            print(f"Class Labels: {class_label}")
            print(f"Regression Labels: {regress_label}")
            
            if isinstance(video_segment, torch.Tensor) and video_segment.nelement() > 0:
                print(f"Video Tensor Shape: {video_segment.shape}")
            else:
                print(f"No video data for segment {j+1}")

# Проверка даталоадера
check_dataloader(train_loader)

            

Loading video: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_820_839_3.mp4
Video duration: 4.01 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])
Loading video: main_dataset_no_aug/train/group_0/2_5395803543229709215_745_763_0.mp4
Video duration: 5.01 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])
Batch 1
Video Segment 1
Class Labels: tensor([2, 2])
Regression Labels: tensor([[0.0000, 4.0100],
        [0.0000, 5.0100]])
Video Tensor Shape: torch.Size([2, 2, 3, 32, 256, 256])
Loading video: main_dataset_no_aug/train/group_0/2_5395803543229709215_780_798_0.mp4
Video duration: 5.01 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])
Loading video: main_dataset_no_aug/train/group_3/2_5395803543229709215_192_365_21.mp4
Video duration: 5.01 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])


In [34]:
# Проверка тренировочного даталоадера
print("Checking training dataloader")
check_dataloader(train_loader, num_batches=2)

# Проверка валидационного даталоадера
print("Checking validation dataloader")
check_dataloader(val_loader, num_batches=2)


Checking training dataloader
Loading video: main_dataset_no_aug/train/group_0/VID-20240301-WA0022_3_21_0.mp4
Video duration: 5.01 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])
Loading video: main_dataset_no_aug/train/group_1/VID_20240304_095352_7_20_1.mp4
Video duration: 5.01 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])
Batch 1
Video Segment 1
Class Labels: tensor([2, 0])
Regression Labels: tensor([[0.0000, 5.0100],
        [0.0000, 5.0100]])
Video Tensor Shape: torch.Size([2, 2, 3, 32, 256, 256])
Loading video: main_dataset_no_aug/train/group_3/2_5395803543229709215_192_365_24.mp4
Video duration: 5.03 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])
Loading video: main_dataset_no_aug/train/group_0/0000000_00000020240221084423_0002_IMP_474_490_1.mp4
Video duration: 5.01 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])
Batch 2
Video Segment 1
Class Labels: tensor([1, 2])
Regression Labels: tensor([[0.0000, 5.0300],
        [0.0000, 

In [36]:
import time

def train_model(model, train_loader, val_loader, num_epochs=25, save_path='model_weights'):
    criterion_class = nn.CrossEntropyLoss()
    criterion_regress = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        # Обучение
        model.train()
        running_loss_class = 0.0
        running_loss_regress = 0.0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = [inp.to(device) for inp in inputs]
            class_labels = torch.cat([label[0].view(-1) for label in labels], dim=0).to(device)
            regress_labels = torch.cat([label[1].view(-1, 2) for label in labels], dim=0).to(device)
            
            optimizer.zero_grad()
            class_out, regress_out = model(inputs)
            loss_class = criterion_class(class_out, class_labels)
            loss_regress = criterion_regress(regress_out, regress_labels)
            loss = loss_class + loss_regress
            loss.backward()
            optimizer.step()
            running_loss_class += loss_class.item()
            running_loss_regress += loss_regress.item()

            # Проверка валидации каждые три итерации
            if (i + 1) % 3 == 0:
                model.eval()
                val_loss_class = 0.0
                val_loss_regress = 0.0
                correct = 0
                total = 0
                
                with torch.no_grad():
                    for val_inputs, val_labels in val_loader:
                        val_inputs = [inp.to(device) for inp in val_inputs]
                        class_labels = torch.cat([label[0].view(-1) for label in val_labels], dim=0).to(device)
                        print(class_labels)
                        regress_labels = torch.cat([label[1].view(-1, 2) for label in val_labels], dim=0).to(device)
                        
                        class_out, regress_out = model(val_inputs)
                        loss_class = criterion_class(class_out, class_labels)
                        loss_regress = criterion_regress(regress_out, regress_labels)
                        val_loss_class += loss_class.item()
                        val_loss_regress += loss_regress.item()
                        
                        _, predicted = torch.max(class_out.data, 1)
                        total += class_labels.size(0)
                        correct += (predicted == class_labels).sum().item()
                
                print(f"Iteration {i+1}/{len(train_loader)} - Validation Loss (Class): {val_loss_class/len(val_loader):.4f}, Loss (Regress): {val_loss_regress/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
                model.train()
        
        epoch_train_time = time.time() - epoch_start_time
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss (Class): {running_loss_class/len(train_loader):.4f}, Loss (Regress): {running_loss_regress/len(train_loader):.4f}")
        print(f"Time for epoch {epoch+1}: {epoch_train_time:.2f} seconds")
        
        # Сохранение весов модели
        torch.save(model.state_dict(), os.path.join(save_path, f'epoch_{epoch+1}.pth'))
    
    print("Training complete")


# Обучение модели
train_model(model, train_loader, val_loader, num_epochs=25)


Loading video: main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_639_649_2.mp4
Error getting duration for video main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_639_649_2.mp4: MoviePy error: failed to read the first frame of video file main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_639_649_2.mp4. That might mean that the file is corrupted. That may also mean that you are using a deprecated version of FFMPEG. On Ubuntu/Debian for instance the version in the repos is deprecated. Please update to a recent version from the website.
Loading video: main_dataset_no_aug/train/group_0/2_5395803543229709215_140_159_2.mp4
Video duration: 5.02 seconds
Video tensor shape: torch.Size([2, 3, 32, 256, 256])


RuntimeError: each element in list of batch should be of equal size

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class SegmentedVideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
        self.class_to_idx = {
            "group_1": 0,  # Нарушение 1
            "group_3": 1,  # Нарушение 3
            "group_0": 2   # Без нарушений
        }
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        print(f"Loading video: {video_path}")
        video = EncodedVideo.from_path(video_path)
        
        # Извлекаем метку класса из пути
        class_label_str = os.path.basename(os.path.dirname(video_path))
        class_label = self.class_to_idx[class_label_str]
        
        # Определяем продолжительность видео
        try:
            video_duration = VideoFileClip(video_path).duration
        except Exception as e:
            # print(f"Error getting duration for video {video_path}: {e}")
            return [], ([], [])
        
        # print(f"Video duration: {video_duration} seconds")
        
        # Интервал от 0 до конца видео
        start_sec = 0
        end_sec = video_duration
        
        # Получаем клип
        try:
            video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
            if self.transform:
                video_data = self.transform(video_data)
            
            video_tensor = video_data["video"]
            
            # Проверка на тип video_tensor и его преобразование
            if isinstance(video_tensor, list):
                # Приведение всех видео к одинаковому размеру кадров
                max_frames = max([vt.shape[1] for vt in video_tensor])
                video_tensor = [torch.nn.functional.pad(vt, (0, 0, 0, 0, 0, max_frames - vt.shape[1])) for vt in video_tensor]
                video_tensor = torch.stack(video_tensor)
                
            regress_label = torch.tensor([start_sec, end_sec], dtype=torch.float32)
            
            # print(f"Video tensor shape: {video_tensor.shape}")
            
            return [video_tensor], ([torch.tensor(class_label, dtype=torch.long)], [regress_label])
        except Exception as e:
            # print(f"Error processing video {video_path}: {e}")
            return [], ([], [])

# Путь к вашим тренировочным видео
train_video_dir = 'main_dataset_no_aug/train'
train_dataset = SegmentedVideoDataset(video_dir=train_video_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Путь к вашим валидационным видео
valid_video_dir = 'main_dataset_no_aug/valid'
valid_dataset = SegmentedVideoDataset(video_dir=valid_video_dir, transform=transform)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True)

# Функция для обучения модели
def train_model(model, train_loader, val_loader, num_epochs=25, save_path='model_weights'):
    criterion_class = nn.CrossEntropyLoss()
    criterion_regress = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        
        # Обучение
        model.train()
        running_loss_class = 0.0
        running_loss_regress = 0.0
        
        for i, (inputs, labels) in enumerate(train_loader):
            if not inputs or not labels:
                continue
            
            print(f"Batch {i+1}, Labels: {labels}")
            if all(len(label) >= 2 for label in labels):
                inputs = [inp.to(device) for inp in inputs]
                class_labels = torch.cat([label[0].view(-1) for label in labels if len(label[0]) > 0], dim=0).to(device)
                regress_labels = torch.cat([label[1].view(-1, 2) for label in labels if len(label[1]) > 0], dim=0).to(device)
                
                optimizer.zero_grad()
                print(f"Inputs: {inputs}")
                class_out, regress_out = model(inputs)
                loss_class = criterion_class(class_out, class_labels)
                loss_regress = criterion_regress(regress_out, regress_labels)
                loss = loss_class + loss_regress
                loss.backward()
                optimizer.step()
                running_loss_class += loss_class.item()
                running_loss_regress += loss_regress.item()

                # Проверка валидации каждые три итерации
                if (i + 1) % 3 == 0:
                    model.eval()
                    val_loss_class = 0.0
                    val_loss_regress = 0.0
                    correct = 0
                    total = 0
                    
                    with torch.no_grad():
                        for val_inputs, val_labels in val_loader:
                            if not val_inputs or not val_labels:
                                continue
                            
                            if all(len(label) >= 2 for label in val_labels):
                                val_inputs = [inp.to(device) for inp in val_inputs]
                                class_labels = torch.cat([label[0].view(-1) for label in val_labels if len(label[0]) > 0], dim=0).to(device)
                                regress_labels = torch.cat([label[1].view(-1, 2) for label in val_labels if len(label[1]) > 0], dim=0).to(device)
                                
                                class_out, regress_out = model(val_inputs)
                                loss_class = criterion_class(class_out, class_labels)
                                loss_regress = criterion_regress(regress_out, regress_labels)
                                val_loss_class += loss_class.item()
                                val_loss_regress += loss_regress.item()
                                
                                _, predicted = torch.max(class_out.data, 1)
                                total += class_labels.size(0)
                                correct += (predicted == class_labels).sum().item()
                    
                    print(f"Iteration {i+1}/{len(train_loader)} - Validation Loss (Class): {val_loss_class/len(val_loader):.4f}, Loss (Regress): {val_loss_regress/len(val_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
                    model.train()
        
        epoch_train_time = time.time() - epoch_start_time
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss (Class): {running_loss_class/len(train_loader):.4f}, Loss (Regress): {running_loss_regress/len(train_loader):.4f}")
        print(f"Time for epoch {epoch+1}: {epoch_train_time:.2f} seconds")
        
        # Сохранение весов модели
        torch.save(model.state_dict(), os.path.join(save_path, f'epoch_{epoch+1}.pth'))
    
    print("Training complete")


# Обучение модели
train_model(model, train_loader, val_loader, num_epochs=25)


In [12]:
import os
import glob
import subprocess
from moviepy.editor import VideoFileClip

def unify_video_ffmpeg(input_path, output_path, target_width=1920, target_height=1080, target_fps=30, target_duration=None):
    try:
        clip = VideoFileClip(input_path)
        
        # Определение текущих параметров видео
        current_duration = clip.duration
        current_width, current_height = clip.size

        # Формирование команды ffmpeg для изменения размеров и частоты кадров
        resize_cmd = [
            'ffmpeg', '-i', input_path, '-vf', 
            f'scale={target_width}:{target_height},fps={target_fps}',
            '-c:v', 'libx264', '-crf', '18', '-preset', 'slow'
        ]
        
        # Добавление параметра продолжительности, если требуется
        if target_duration is not None:
            resize_cmd.extend(['-t', str(target_duration)])

        resize_cmd.append(output_path)
        
        # Выполнение команды ffmpeg
        subprocess.run(resize_cmd, check=True)
        print(f"Processed video saved to {output_path}")
    except Exception as e:
        print(f"Error processing video {input_path}: {e}")

def unify_all_videos_ffmpeg(video_dir, output_dir, target_width=1920, target_height=1080, target_fps=30, target_duration=None):
    video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
    for video_path in video_paths:
        relative_path = os.path.relpath(video_path, video_dir)
        output_path = os.path.join(output_dir, relative_path)
        output_dirname = os.path.dirname(output_path)

        if not os.path.exists(output_dirname):
            os.makedirs(output_dirname)

        unify_video_ffmpeg(video_path, output_path, target_width, target_height, target_fps, target_duration)

# Пример использования
input_video_dir = 'main_dataset_no_aug/train'
output_video_dir = 'main_dataset_no_aug/train_unified'
target_width = 1280
target_height = 720
target_fps = 30
target_duration = None  # Можно указать None, если не требуется унификация по продолжительности

unify_all_videos_ffmpeg(input_video_dir, output_video_dir, target_width, target_height, target_fps, target_duration)


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

Processed video saved to main_dataset_no_aug/train_unified/group_3/0000000_00000020240221085923_0003_IMP_552_718_19.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

Processed video saved to main_dataset_no_aug/train_unified/group_3/2_5395803543229709215_192_365_21.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

...

Processed video saved to main_dataset_no_aug/train_unified/group_1/0000000_00000020240508114229_0002_333_335_0.mp4


[out#0/mp4 @ 0x6000001283c0] video:2613kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.081175%
frame=  150 fps=108 q=-1.0 Lsize=    2615kB time=00:00:04.90 bitrate=4372.0kbits/s speed=3.52x    
[libx264 @ 0x125707650] frame I:2     Avg QP:17.22  size: 52740
[libx264 @ 0x125707650] frame P:94    Avg QP:17.92  size: 20780
[libx264 @ 0x125707650] frame B:54    Avg QP:18.11  size: 11412
[libx264 @ 0x125707650] consecutive B-frames: 47.3% 10.7% 10.0% 32.0%
[libx264 @ 0x125707650] mb I  I16..4: 21.9% 63.8% 14.3%
[libx264 @ 0x125707650] mb P  I16..4: 16.3% 26.1%  2.1%  P16..4: 30.0%  7.7%  2.8%  0.0%  0.0%    skip:15.1%
[libx264 @ 0x125707650] mb B  I16..4:  5.2%  6.2%  0.4%  B16..8: 34.6%  7.6%  0.9%  direct:12.6%  skip:32.6%  L0:63.5% L1:32.5% BI: 4.0%
[libx264 @ 0x125707650] 8x8 transform intra:58.2% inter:64.8%
[libx264 @ 0x125707650] direct mvs  spatial:96.3% temporal:3.7%
[libx264 @ 0x125707650] coded y,uvDC,uvAC intra: 33.2% 71.7% 21.0% inter: 20.9% 37

Processed video saved to main_dataset_no_aug/train_unified/group_1/2_5395803543229709828_279_300_0.mp4


Processed video saved to main_dataset_no_aug/train_unified/group_1/2_5395803543229709828_279_300_1.mp4


[out#0/mp4 @ 0x6000010043c0] video:2242kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.109918%
frame=  150 fps=116 q=-1.0 Lsize=    2245kB time=00:00:04.90 bitrate=3753.1kbits/s speed=3.78x    
[libx264 @ 0x121f07650] frame I:2     Avg QP:16.81  size: 32840
[libx264 @ 0x121f07650] frame P:57    Avg QP:17.65  size: 22381
[libx264 @ 0x121f07650] frame B:91    Avg QP:18.73  size: 10485
[libx264 @ 0x121f07650] consecutive B-frames: 16.7%  2.7% 14.0% 66.7%
[libx264 @ 0x121f07650] mb I  I16..4: 28.1% 63.7%  8.2%
[libx264 @ 0x121f07650] mb P  I16..4: 14.6% 24.4%  1.9%  P16..4: 31.4%  9.1%  2.5%  0.0%  0.0%    skip:16.1%
[libx264 @ 0x121f07650] mb B  I16..4:  2.2%  2.6%  0.2%  B16..8: 41.4%  8.3%  1.0%  direct:10.3%  skip:33.9%  L0:56.7% L1:37.5% BI: 5.8%
[libx264 @ 0x121f07650] 8x8 transform intra:58.7% inter:70.2%
[libx264 @ 0x121f07650] direct mvs  spatial:92.3% temporal:7.7%
[libx264 @ 0x121f07650] coded y,uvDC,uvAC intra: 40.2% 70.5% 17.8% inter: 20.6% 32

In [10]:
import os
import glob
import cv2

def get_video_file_sizes(video_dir):
    video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
    video_sizes = {}
    
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Failed to open video: {video_path}")
            continue
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        duration = frame_count / fps if fps > 0 else 0
        video_sizes[video_path] = (width, height, frame_count, fps, duration)
        cap.release()
    
    return video_sizes

video_dir = 'main_dataset_no_aug/train'  # Измените на путь к вашей директории
video_sizes = get_video_file_sizes(video_dir)

for video_path, size_info in video_sizes.items():
    print(f"Video: {video_path}")
    print(f"Width: {size_info[0]}, Height: {size_info[1]}, Frame Count: {size_info[2]}, FPS: {size_info[3]}, Duration: {size_info[4]:.2f} seconds")
    print()


Video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718_19.mp4
Width: 1920, Height: 1080, Frame Count: 182, FPS: 29.97002997002997, Duration: 6.07 seconds

Video: main_dataset_no_aug/train/group_3/2_5395803543229709215_192_365_21.mp4
Width: 1280, Height: 720, Frame Count: 178, FPS: 29.97002997002997, Duration: 5.94 seconds

Video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718_31.mp4
Width: 1920, Height: 1080, Frame Count: 156, FPS: 29.97002997002997, Duration: 5.21 seconds

Video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718_25.mp4
Width: 1920, Height: 1080, Frame Count: 153, FPS: 29.97002997002997, Duration: 5.11 seconds

Video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718_4.mp4
Width: 1920, Height: 1080, Frame Count: 174, FPS: 29.97002997002997, Duration: 5.81 seconds

Video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_

In [4]:
import os
import glob
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip
import subprocess

def unify_video_ffmpeg(input_path, output_path, target_width=1920, target_height=1080, target_fps=30, target_duration=None):
    try:
        clip = VideoFileClip(input_path)
        
        # Определение текущих параметров видео
        current_duration = clip.duration
        current_width, current_height = clip.size

        # Формирование команды ffmpeg для изменения размеров и частоты кадров
        resize_cmd = [
            'ffmpeg', '-i', input_path, '-vf', 
            f'scale={target_width}:{target_height},fps={target_fps}',
            '-c:v', 'libx264', '-crf', '18', '-preset', 'slow'
        ]
        
        # Добавление параметра продолжительности, если требуется
        if target_duration is not None:
            if current_duration < target_duration:
                resize_cmd.extend(['-t', str(target_duration)])
            else:
                resize_cmd.extend(['-t', str(target_duration)])

        resize_cmd.append(output_path)
        
        # Выполнение команды ffmpeg
        subprocess.run(resize_cmd, check=True)
        print(f"Processed video saved to {output_path}")
    except Exception as e:
        print(f"Error processing video {input_path}: {e}")

def unify_all_videos_ffmpeg(video_dir, output_dir, target_width=1920, target_height=1080, target_fps=30, target_duration=None):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
    for video_path in video_paths:
        video_name = os.path.basename(video_path)
        output_path = os.path.join(output_dir, video_name)
        unify_video_ffmpeg(video_path, output_path, target_width, target_height, target_fps, target_duration)

# Пример использования
input_video_dir = 'main_dataset_no_aug/valid'
output_video_dir = 'main_dataset_no_aug/valid_unified'
target_width = 1280
target_height = 720
target_fps = 30
target_duration = None  # Можно указать None, если не требуется унификация по продолжительности

unify_all_videos_ffmpeg(input_video_dir, output_video_dir, target_width, target_height, target_fps, target_duration)


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

Processed video saved to main_dataset_no_aug/valid_unified/2_5395803543229709215_192_365_21.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

Processed video saved to main_dataset_no_aug/valid_unified/2_5395803543229709215_192_365_34.mp4


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

...

[out#0/mp4 @ 0x6000027dc840] video:4846kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.054652%
frame=  150 fps= 81 q=-1.0 Lsize=    4849kB time=00:00:04.90 bitrate=8106.1kbits/s speed=2.64x    
[libx264 @ 0x11f608ab0] frame I:1     Avg QP:19.88  size:132745
[libx264 @ 0x11f608ab0] frame P:38    Avg QP:19.39  size: 83583
[libx264 @ 0x11f608ab0] frame B:111   Avg QP:24.80  size: 14889
[libx264 @ 0x11f608ab0] consecutive B-frames:  0.7%  1.3%  2.0% 96.0%
[libx264 @ 0x11f608ab0] mb I  I16..4:  4.9% 70.2% 24.9%
[libx264 @ 0x11f608ab0] mb P  I16..4:  1.4%  4.9%  1.0%  P16..4: 43.9% 30.8% 16.8%  0.0%  0.0%    skip: 1.1%
[libx264 @ 0x11f608ab0] mb B  I16..4:  0.3%  0.3%  0.0%  B16..8: 60.0%  9.8%  3.0%  direct: 5.1%  skip:21.5%  L0:58.5% L1:26.0% BI:15.5%
[libx264 @ 0x11f608ab0] 8x8 transform intra:64.8% inter:23.8%
[libx264 @ 0x11f608ab0] direct mvs  spatial:92.8% temporal:7.2%
[libx264 @ 0x11f608ab0] coded y,uvDC,uvAC intra: 66.3% 88.0% 55.6% inter: 33.3% 25

Processed video saved to main_dataset_no_aug/valid_unified/2_5395803543229709215_745_763_1.mp4


[out#0/mp4 @ 0x600000500480] video:2245kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.104554%
frame=  151 fps=135 q=-1.0 Lsize=    2248kB time=00:00:04.93 bitrate=3732.5kbits/s speed=4.42x    
[libx264 @ 0x124f07650] frame I:1     Avg QP:19.80  size:122492
[libx264 @ 0x124f07650] frame P:67    Avg QP:18.77  size: 26622
[libx264 @ 0x124f07650] frame B:83    Avg QP:21.33  size:  4728
[libx264 @ 0x124f07650] consecutive B-frames: 22.5%  7.9% 13.9% 55.6%
[libx264 @ 0x124f07650] mb I  I16..4:  4.6% 64.1% 31.3%
[libx264 @ 0x124f07650] mb P  I16..4:  2.8%  7.7%  2.0%  P16..4: 39.7% 14.6%  7.5%  0.0%  0.0%    skip:25.7%
[libx264 @ 0x124f07650] mb B  I16..4:  0.2%  0.3%  0.1%  B16..8: 28.5%  5.0%  1.1%  direct: 0.7%  skip:64.2%  L0:58.8% L1:31.7% BI: 9.6%
[libx264 @ 0x124f07650] 8x8 transform intra:61.3% inter:44.6%
[libx264 @ 0x124f07650] direct mvs  spatial:90.4% temporal:9.6%
[libx264 @ 0x124f07650] coded y,uvDC,uvAC intra: 56.3% 83.0% 32.1% inter: 14.5% 15

Processed video saved to main_dataset_no_aug/valid_unified/2_5395803543229709215_745_763_3.mp4



KeyboardInterrupt



In [2]:
import os
import subprocess
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip, ImageSequenceClip

def get_total_frames(video_path):
    print(f"Getting total frames for: {video_path}")
    video_info_cmd = [
        'ffprobe', '-v', 'error', '-select_streams', 'v:0', '-count_frames',
        '-show_entries', 'stream=nb_read_frames', '-of', 'default=nokey=1:noprint_wrappers=1', video_path
    ]
    result = subprocess.run(video_info_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    total_frames = result.stdout.strip()
    if not total_frames.isdigit():
        raise ValueError(f"Cannot get total frames for: {video_path}")
    print(f"Total frames: {total_frames}")
    return int(total_frames)

def get_fps(video_path):
    print(f"Getting FPS for: {video_path}")
    fps_cmd = [
        'ffprobe', '-v', '0', '-select_streams', 'v:0', '-show_entries', 'stream=r_frame_rate',
        '-of', 'csv=p=0', video_path
    ]
    result = subprocess.run(fps_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    fps = eval(result.stdout.strip())
    print(f"FPS: {fps}")
    return fps

def pad_video_to_duration(video_path, target_duration, fps):
    print(f"Padding video: {video_path} to duration: {target_duration} seconds")
    clip = VideoFileClip(video_path)
    duration = clip.duration
    
    if duration < target_duration:
        # Calculate the number of frames needed to pad
        total_frames = int(fps * target_duration)
        current_frames = int(fps * duration)
        frames_to_pad = total_frames - current_frames
        
        # Duplicate the last frame to pad the video
        last_frame = clip.get_frame(duration)
        padded_frames = [last_frame] * frames_to_pad
        
        # Create new video with padding
        original_frames = list(clip.iter_frames(fps=fps))
        video_array = original_frames + padded_frames
        padded_clip = ImageSequenceClip(video_array, fps=fps)
        padded_clip.write_videofile(video_path, codec='libx264')
        
    clip.close()

def split_video(video_path, chunk_duration=5):
    print(f"Splitting video: {video_path} into chunks of {chunk_duration} seconds")
    fps = get_fps(video_path)
    total_frames = get_total_frames(video_path)
    total_duration = total_frames / fps
    
    video_basename = os.path.basename(video_path)
    video_dir = os.path.dirname(video_path)
    output_dir = os.path.join(video_dir, "split_videos")
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    start = 0
    chunk_idx = 0
    
    while start < total_duration:
        end = min(start + chunk_duration, total_duration)
        output_file = os.path.join(output_dir, f"{os.path.splitext(video_basename)[0]}_{chunk_idx}.mp4")
        print(f"Extracting subclip: {start} to {end}, output file: {output_file}")
        ffmpeg_extract_subclip(video_path, start, end, targetname=output_file)
        
        if end - start < chunk_duration:
            pad_video_to_duration(output_file, chunk_duration, fps)
        
        start = end
        chunk_idx += 1
    
    print(f"Removing original video: {video_path}")
    os.remove(video_path)

def process_directory(directory, chunk_duration=5):
    for root, dirs, files in os.walk(directory):
        if 'split_videos' in root:
            continue  # Skip already processed directories
        for file in files:
            if file.endswith('.mp4'):
                video_path = os.path.join(root, file)
                print(f"Processing video: {video_path}")
                try:
                    split_video(video_path, chunk_duration)
                except Exception as e:
                    print(f"Error processing video {video_path}: {e}")

# Пример использования функции
base_directory_path = 'main_dataset_no_aug'

# Обработка тренировочного набора
train_directory_path = os.path.join(base_directory_path, 'train')
process_directory(train_directory_path, chunk_duration=5)

# Обработка валидационного набора
valid_directory_path = os.path.join(base_directory_path, 'valid')
process_directory(valid_directory_path, chunk_duration=5)


Processing video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718.mp4
Splitting video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718.mp4
Total frames: 4975
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/train/group_3/split_videos/0000000_00000020240221085923_0003_IMP_552_718_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/train/group_3/split_videos/0000000_00000020240221085923_0003_IMP_552_718_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 10 to 15, output file: main_dataset_no_aug/train/group_3/split_videos/0000000_00

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/train/group_3/split_videos/0000000_00000020240221085923_0003_IMP_552_718_33.mp4
Removing original video: main_dataset_no_aug/train/group_3/0000000_00000020240221085923_0003_IMP_552_718.mp4
Processing video: main_dataset_no_aug/train/group_3/0000000_00000020240221082923_0001_IMP_370_380.mp4
Splitting video: main_dataset_no_aug/train/group_3/0000000_00000020240221082923_0001_IMP_370_380.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/train/group_3/0000000_00000020240221082923_0001_IMP_370_380.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/train/group_3/0000000_00000020240221082923_0001_IMP_370_380.mp4
Total frames: 300
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/train/group_3/split_videos/0000000_00000020240221082923_0001_IMP_370_380_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/t

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/train/group_0/split_videos/VID_20240304_095352_20_36_0.mp4
Removing original video: main_dataset_no_aug/train/group_0/VID_20240304_095352_20_36.mp4
Processing video: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_542_552.mp4
Splitting video: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_542_552.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_542_552.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_542_552.mp4
Total frames: 300
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/train/group_0/split_videos/0000000_00000020240221085923_0003_IMP_542_552_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/train/group_0/split_videos/0000000_0000002

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/train/group_0/split_videos/0000000_00000020240221085923_0003_IMP_802_818_3.mp4
Removing original video: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_802_818.mp4
Processing video: main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_359_370.mp4
Splitting video: main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_359_370.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_359_370.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_359_370.mp4
Total frames: 329
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/train/group_0/split_videos/0000000_00000020240221082923_0001_IMP_359_370_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/tr

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/train/group_0/split_videos/0000000_00000020240221082923_0001_IMP_359_370_2.mp4
Removing original video: main_dataset_no_aug/train/group_0/0000000_00000020240221082923_0001_IMP_359_370.mp4
Processing video: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_820_839.mp4
Splitting video: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_820_839.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_820_839.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/train/group_0/0000000_00000020240221085923_0003_IMP_820_839.mp4
Total frames: 569
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/train/group_0/split_videos/0000000_00000020240221085923_0003_IMP_820_839_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/tr

...

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_155_166_2.mp4
Removing original video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_155_166.mp4
Processing video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_365_379.mp4
Splitting video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_365_379.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/valid/group_0/2_5395803543229709215_365_379.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/valid/group_0/2_5395803543229709215_365_379.mp4
Total frames: 419
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_365_379_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_365_379_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - C

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709261_107_118_2.mp4
Removing original video: main_dataset_no_aug/valid/group_0/2_5395803543229709261_107_118.mp4
Processing video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_780_798.mp4
Splitting video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_780_798.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/valid/group_0/2_5395803543229709215_780_798.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/valid/group_0/2_5395803543229709215_780_798.mp4
Total frames: 540
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_780_798_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_780_798_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - C

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_780_798_3.mp4
Removing original video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_780_798.mp4
Processing video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_168_183.mp4
Splitting video: main_dataset_no_aug/valid/group_0/2_5395803543229709215_168_183.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/valid/group_0/2_5395803543229709215_168_183.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/valid/group_0/2_5395803543229709215_168_183.mp4
Total frames: 450
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_168_183_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/valid/group_0/split_videos/2_5395803543229709215_168_183_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - C

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/valid/group_1/split_videos/2_5395803543229709215_166_168_0.mp4
Removing original video: main_dataset_no_aug/valid/group_1/2_5395803543229709215_166_168.mp4
Processing video: main_dataset_no_aug/valid/group_1/2_5395803543229709215_763_780.mp4
Splitting video: main_dataset_no_aug/valid/group_1/2_5395803543229709215_763_780.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/valid/group_1/2_5395803543229709215_763_780.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/valid/group_1/2_5395803543229709215_763_780.mp4
Total frames: 509
Extracting subclip: 0 to 5, output file: main_dataset_no_aug/valid/group_1/split_videos/2_5395803543229709215_763_780_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Extracting subclip: 5 to 10, output file: main_dataset_no_aug/valid/group_1/split_videos/2_5395803543229709215_763_780_1.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - C

Moviepy - Done !
Moviepy - video ready main_dataset_no_aug/valid/group_1/split_videos/2_5395803543229709215_763_780_3.mp4
Removing original video: main_dataset_no_aug/valid/group_1/2_5395803543229709215_763_780.mp4
Processing video: main_dataset_no_aug/valid/group_1/2_5395803543229709261_103_107.mp4
Splitting video: main_dataset_no_aug/valid/group_1/2_5395803543229709261_103_107.mp4 into chunks of 5 seconds
Getting FPS for: main_dataset_no_aug/valid/group_1/2_5395803543229709261_103_107.mp4
FPS: 29.97002997002997
Getting total frames for: main_dataset_no_aug/valid/group_1/2_5395803543229709261_103_107.mp4
Total frames: 120
Extracting subclip: 0 to 4.0040000000000004, output file: main_dataset_no_aug/valid/group_1/split_videos/2_5395803543229709261_103_107_0.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Padding video: main_dataset_no_aug/valid/group_1/split_videos/2_5395803543229709261_103_107_0.mp4 to duration: 5 seconds
Error processing video main_dataset_no

In [ ]:
import time
import torch
import json
import urllib
from typing import Dict
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
import os
import glob
import numpy as np


In [8]:
# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Three outputs for multi-class classification
model = model.to(device)

# Путь к контрольной точке
checkpoint_path = 'slowfast_epoch_5.pth'

# Загрузка контрольной точки
checkpoint = torch.load(checkpoint_path)

# Восстановление весов модели
model.load_state_dict(checkpoint['model_state_dict'])

# Восстановление состояния оптимизатора
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Восстановление других параметров, если необходимо
start_epoch = checkpoint['epoch']
loss = checkpoint['loss']


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

In [ ]:
# Длительность клипа
clip_duration = (num_frames * sampling_rate) / frames_per_second

class VideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
        self.class_to_idx = {
            "group_1": 0,  # Нарушение 1
            "group_3": 1,  # Нарушение 3
            "group_remaining": 2  # Без нарушений
        }
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video = EncodedVideo.from_path(video_path)
        start_sec = 0
        end_sec = (num_frames * sampling_rate) / frames_per_second
        video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
        if self.transform:
            video_data = self.transform(video_data)
        video_tensor = video_data["video"]
        class_name = os.path.basename(os.path.dirname(video_path))
        label = self.class_to_idx[class_name]
        return video_tensor, torch.tensor(label, dtype=torch.long)

# Путь к вашим тренировочным видео
train_video_dir = 'resnet_dataset/train'
train_dataset = VideoDataset(video_dir=train_video_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Путь к вашим валидационным видео
valid_video_dir = 'resnet_dataset/valid'
valid_dataset = VideoDataset(video_dir=valid_video_dir, transform=transform)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True)


In [ ]:
# Функция для предсказаний
def predict(video_dir, model, transform, device, batch_size=2):
    dataset = VideoDataset(video_dir=video_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = [i.to(device) for i in inputs]
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions

# Пример использования функции предсказаний
valid_video_dir = 'resnet_dataset/valid'
predictions = predict(valid_video_dir, model, transform, device)
print(predictions)


# Инференс 

In [11]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Three outputs for multi-class classification
model = model.to(device)

# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform = ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x / 255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=side_size),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# Длительность клипа
clip_duration = (num_frames * sampling_rate) / frames_per_second

class VideoDataset(Dataset):
    def __init__(self, video_dir, transform=None):
        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        video = EncodedVideo.from_path(video_path)
        start_sec = 0
        end_sec = (num_frames * sampling_rate) / frames_per_second
        video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
        if self.transform:
            video_data = self.transform(video_data)
        video_tensor = video_data["video"]
        return video_tensor, video_path

def run_inference(video_dir: str, checkpoint_path: str, batch_size: int = 2):
    # Загрузка контрольной точки
    checkpoint = torch.load(checkpoint_path)

    # Восстановление весов модели
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    dataset = VideoDataset(video_dir=video_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    predictions = []

    with torch.no_grad():
        for inputs, video_paths in loader:
            inputs = [i.to(device) for i in inputs]
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(list(zip(video_paths, preds.cpu().numpy())))

    return predictions

# Пример использования функции run_inference
checkpoint_path = 'slowfast_epoch_5.pth'
valid_video_dir = 'testing'
predictions = run_inference(valid_video_dir, checkpoint_path)
print(predictions)


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


[('testing/0000000_00000020240508132920_0006_1.mp4', 0), ('testing/0000000_00000020240221084423_0002_IMP_0.mp4', 2), ('testing/2_5395803543229709828_0.mp4', 0)]


In [ ]:
            "group_1": 0,  # Нарушение 1
            "group_3": 1,  # Нарушение 3
            "group_remaining": 2  # Без нарушений

In [36]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.9 MB/s eta 0:00:0000:0100:010m
DEPRECATION: lightning-lite 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of lightning-lite or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.8.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [40]:
import os
import torch
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda, Normalize, Resize, CenterCrop
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as ReportLabImage, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd
import io

# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=False)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Три выхода для многоклассовой классификации
model = model.to(device)

# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

# Преобразования для видео
class VideoTransform:
    def __init__(self, side_size, mean, std, crop_size):
        self.transforms = Compose([
            Lambda(lambda x: x / 255.0),
            Resize((side_size, side_size)),
            CenterCrop(crop_size),
            Lambda(lambda x: self.normalize_video(x, mean, std))
        ])

    def __call__(self, x):
        return self.transforms(x)

    def normalize_video(self, video, mean, std):
        mean = torch.tensor(mean).view(3, 1, 1, 1)
        std = torch.tensor(std).view(3, 1, 1, 1)
        return (video - mean) / std

transform = VideoTransform(side_size, mean, std, crop_size)

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

def load_and_preprocess_video(video_path, transform, num_frames=32, sampling_rate=2):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(num_frames):
        frame_id = i * sampling_rate
        if frame_id >= frame_count:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    if not frames:
        raise ValueError(f"Не удалось загрузить кадры из видео: {video_path}")

    if len(frames) < num_frames:
        # Повторение последнего кадра, если видео короче ожидаемого числа кадров
        frames += [frames[-1]] * (num_frames - len(frames))

    frames = np.stack(frames)
    frames = torch.from_numpy(frames).permute(3, 0, 1, 2)  # Преобразование в CTHW
    frames = transform(frames)  # Применение преобразований

    pack_pathway = PackPathway()
    video_tensor = pack_pathway(frames)
    return video_tensor

# Словарь для преобразования меток в текстовые описания
label_to_description = {
    0: "Нарушение: Работы под составом",
    1: "Нарушение: Передвижение на подвижном составе",
    2: "Без нарушений"
}

def run_inference(video_path: str, checkpoint_path: str):
    # Загрузка контрольной точки
    checkpoint = torch.load(checkpoint_path)

    # Восстановление весов модели
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    video_tensor = load_and_preprocess_video(video_path, transform)
    video_tensor = [i.unsqueeze(0).to(device) for i in video_tensor]  # Добавление batch dimension

    with torch.no_grad():
        outputs = model(video_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        pred = torch.argmax(outputs, dim=1).item()
        description = label_to_description[pred]

    return video_path, description, probabilities.squeeze().cpu().numpy()

# Пример использования функции run_inference
checkpoint_path = 'slowfast_epoch_5.pth'
video_path = 'testing/0000000_00000020240221084423_0002_IMP_0.mp4'
prediction = run_inference(video_path, checkpoint_path)
print(prediction)

# Функция для создания отчета
def create_report(violations, report_path):
    doc = SimpleDocTemplate(report_path, pagesize=letter)
    elements = []
    styles = getSampleStyleSheet()
    normal_style = styles['Normal']

    intro_text = ("Этот отчет предоставляет обзор нарушений безопасности, обнаруженных в анализируемых видео. "
                  "Основная цель - выявить области для улучшения соблюдения требований безопасности. "
                  "Диаграмма ниже показывает частоту различных типов нарушений.")
    elements.append(Paragraph(intro_text, normal_style))
    elements.append(Spacer(1, 20))

    for violation in violations:
        file_text = f"Файл: {violation[0]}"
        violation_text = f"Нарушение: {violation[1]}"
        probabilities_text = f"Вероятности: {violation[2]}"

        text_data = [
            Paragraph(file_text, normal_style),
            Paragraph(violation_text, normal_style),
            Paragraph(probabilities_text, normal_style)
        ]

        for paragraph in text_data:
            elements.append(paragraph)
        elements.append(Spacer(1, 20))

    doc.build(elements)

# Пример использования функции create_report
report_path = 'violations_report.pdf'
create_report([prediction], report_path)

# Функция для сохранения данных в Excel
def save_to_excel(violations, excel_path):
    # Создаем DataFrame только с полями 'file' и 'violation'
    df = pd.DataFrame(violations, columns=['file', 'violation'])
    df.to_excel(excel_path, index=False)

# Пример использования функции save_to_excel
excel_path = 'violations_report.xlsx'
save_to_excel([prediction[:2]], excel_path)  # Сохраняем только файл и тип нарушения


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


('testing/0000000_00000020240221084423_0002_IMP_0.mp4', 'Без нарушений', array([0.03243762, 0.01094951, 0.9566129 ], dtype=float32))


In [29]:
import os
import glob 
import torch
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda, Normalize, Resize, CenterCrop

# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели с проверкой
try:
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
    model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Three outputs for multi-class classification
    model = model.to(device)
except Exception as e:
    raise RuntimeError(f"Ошибка при загрузке модели: {e}")

# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4
interval_duration = 10  # Duration of each interval in seconds

# Преобразования для видео
class VideoTransform:
    def __init__(self, side_size, mean, std, crop_size):
        self.transforms = Compose([
            Lambda(lambda x: x / 255.0),
            Resize((side_size, side_size)),
            CenterCrop(crop_size),
            Lambda(lambda x: self.normalize_video(x, mean, std))
        ])

    def __call__(self, x):
        return self.transforms(x)

    def normalize_video(self, video, mean, std):
        mean = torch.tensor(mean).view(3, 1, 1, 1)
        std = torch.tensor(std).view(3, 1, 1, 1)
        return (video - mean) / std

transform = VideoTransform(side_size, mean, std, crop_size)

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        # Ensure the sizes match
        if slow_pathway.shape[1] != fast_pathway.shape[1] // slowfast_alpha:
            min_length = min(slow_pathway.shape[1], fast_pathway.shape[1] // slowfast_alpha)
            slow_pathway = slow_pathway[:, :min_length, :, :, :]
            fast_pathway = fast_pathway[:, :min_length * slowfast_alpha, :, :, :]

        frame_list = [slow_pathway, fast_pathway]
        return frame_list

class VideoDataset(Dataset):
    def __init__(self, video_dir, transform=None, num_frames=32, sampling_rate=2, interval_duration=10):
        if not os.path.isdir(video_dir):
            raise ValueError(f"Директория видео не найдена: {video_dir}")

        self.video_dir = video_dir
        self.transform = transform
        self.video_paths = glob.glob(os.path.join(video_dir, '**/*.mp4'), recursive=True)
        if len(self.video_paths) == 0:
            raise ValueError(f"В директории {video_dir} не найдено видео файлов.")
            
        self.num_frames = num_frames
        self.sampling_rate = sampling_rate
        self.interval_duration = interval_duration
    
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        if idx >= len(self.video_paths):
            raise IndexError(f"Индекс {idx} выходит за пределы доступных видео файлов.")

        video_path = self.video_paths[idx]
        intervals = self._split_video_into_intervals(video_path)
        video_tensors = [self._process_interval(video_path, interval) for interval in intervals]
        return video_tensors, video_path, intervals
    
    def _split_video_into_intervals(self, path):
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            raise IOError(f"Не удалось открыть видео файл: {path}")

        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count / fps

        intervals = []
        start_frame = 0
        end_frame = int(fps * self.interval_duration)

        while start_frame < frame_count:
            intervals.append((start_frame, min(end_frame, frame_count)))
            start_frame = end_frame
            end_frame += int(fps * self.interval_duration)

        cap.release()
        return intervals
    
    def _process_interval(self, path, interval):
        start_frame, end_frame = interval
        cap = cv2.VideoCapture(path)
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        frames = []
        
        for i in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
        
        cap.release()
        if len(frames) == 0:
            raise ValueError(f"Не удалось прочитать кадры из интервала {interval} в видео {path}.")

        frames = np.stack(frames)
        frames = torch.from_numpy(frames).permute(3, 0, 1, 2)  # Convert to CTHW

        if self.transform:
            frames = self.transform(frames)

        pack_pathway = PackPathway()
        video_tensor = pack_pathway(frames)
        return video_tensor

def run_inference(video_dir: str, checkpoint_path: str, batch_size: int = 2):
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Файл контрольной точки не найден: {checkpoint_path}")

    # Загрузка контрольной точки
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if 'model_state_dict' not in checkpoint:
        raise ValueError("В контрольной точке отсутствует ключ 'model_state_dict'.")

    # Восстановление весов модели
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    dataset = VideoDataset(video_dir=video_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    predictions = []

    with torch.no_grad():
        for video_tensors, video_path, intervals in loader:
            for i, video_tensor in enumerate(video_tensors):
                inputs = [v.to(device) for v in video_tensor]
                
                # Ensure inputs sizes match
                min_length = min(inputs[0].shape[1], inputs[1].shape[1])
                inputs[0] = inputs[0][:, :min_length, :, :, :]
                inputs[1] = inputs[1][:, :min_length, :, :, :]
                
                if len(inputs) == 0:
                    raise ValueError(f"Пустой входной тензор для видео {video_path} в интервале {intervals[i]}.")

                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                
                start_frame, end_frame = intervals[i]
                start_time = start_frame / frames_per_second
                end_time = end_frame / frames_per_second
                start_min_sec = f"{int(start_time // 60)}:{int(start_time % 60):02d}"
                end_min_sec = f"{int(end_time // 60)}:{int(end_time % 60):02d}"
                interval_str = f"{start_min_sec}-{end_min_sec}"
                
                predictions.append((video_path, interval_str, preds.cpu().numpy()))

    return predictions

# Пример использования функции run_inference
checkpoint_path = 'slowfast_epoch_5.pth'
valid_video_dir = 'testing'
try:
    predictions = run_inference(valid_video_dir, checkpoint_path)
    for video_path, interval_str, pred in predictions:
        print(f"Video: {video_path}, Interval: {interval_str}, Prediction: {pred}")
except Exception as e:
    print(f"Ошибка выполнения: {e}")


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


Ошибка выполнения: Sizes of tensors must match except in dimension 1. Expected size 37 but got size 38 for tensor number 1 in the list.


In [45]:
import os
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def split_video(video_path, segment_length=10):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"The video file at {video_path} does not exist.")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file at {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    segment_paths = []
    
    for start_time in range(0, int(duration), segment_length):
        end_time = start_time + segment_length
        if end_time > duration:
            end_time = duration
        segment_path = f"{video_path.split('.')[0]}_segment_{start_time}_{end_time}.mp4"
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=segment_path)
        segment_paths.append(segment_path)
    
    cap.release()
    return segment_paths

def run_inference_on_segments(segment_paths, checkpoint_path):
    # Assuming run_inference is defined elsewhere and works with single video file paths
    predictions = []
    for segment_path in segment_paths:
        prediction = run_inference(segment_path, checkpoint_path)
        print(prediction)
        predictions.append(prediction)
    return predictions

# Example usage:
video_path = 'testing/0000000_00000020240508132920_0006_1.mp4'
checkpoint_path = 'slowfast_epoch_5.pth'

try:
    segment_paths = split_video(video_path)
    predictions = run_inference_on_segments(segment_paths, checkpoint_path)
except FileNotFoundError as e:
    print(e)
except ValueError as e:
    print(e)


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


('testing/0000000_00000020240508132920_0006_1_segment_0_5.005.mp4', 'Нарушение: Работы под составом', array([9.9626714e-01, 8.6023146e-04, 2.8726631e-03], dtype=float32))


In [43]:
import os
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def split_video(video_path, segment_length=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    segment_paths = []
    
    for start_time in range(0, int(duration), segment_length):
        end_time = start_time + segment_length
        if end_time > duration:
            end_time = duration
        segment_path = f"{video_path.split('.')[0]}_segment_{start_time}_{end_time}.mp4"
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=segment_path)
        segment_paths.append(segment_path)
    
    cap.release()
    return segment_paths

def run_inference_on_segments(segment_paths, checkpoint_path):
    predictions = []
    for segment_path in segment_paths:
        prediction = run_inference(segment_path, checkpoint_path)
        predictions.append(prediction)
    return predictions

# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=False)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Три выхода для многоклассовой классификации
model = model.to(device)

# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

# Преобразования для видео
class VideoTransform:
    def __init__(self, side_size, mean, std, crop_size):
        self.transforms = Compose([
            Lambda(lambda x: x / 255.0),
            Resize((side_size, side_size)),
            CenterCrop(crop_size),
            Lambda(lambda x: self.normalize_video(x, mean, std))
        ])

    def __call__(self, x):
        return self.transforms(x)

    def normalize_video(self, video, mean, std):
        mean = torch.tensor(mean).view(3, 1, 1, 1)
        std = torch.tensor(std).view(3, 1, 1, 1)
        return (video - mean) / std

transform = VideoTransform(side_size, mean, std, crop_size)

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

def load_and_preprocess_video(video_path, transform, num_frames=32, sampling_rate=2):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in range(num_frames):
        frame_id = i * sampling_rate
        if frame_id >= frame_count:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    if not frames:
        raise ValueError(f"Не удалось загрузить кадры из видео: {video_path}")

    if len(frames) < num_frames:
        # Повторение последнего кадра, если видео короче ожидаемого числа кадров
        frames += [frames[-1]] * (num_frames - len(frames))

    frames = np.stack(frames)
    frames = torch.from_numpy(frames).permute(3, 0, 1, 2)  # Преобразование в CTHW
    frames = transform(frames)  # Применение преобразований

    pack_pathway = PackPathway()
    video_tensor = pack_pathway(frames)
    return video_tensor

# Словарь для преобразования меток в текстовые описания
label_to_description = {
    0: "Нарушение: Работы под составом",
    1: "Нарушение: Передвижение на подвижном составе",
    2: "Без нарушений"
}

def run_inference(video_path: str, checkpoint_path: str):
    # Загрузка контрольной точки
    checkpoint = torch.load(checkpoint_path)

    # Восстановление весов модели
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    video_tensor = load_and_preprocess_video(video_path, transform)
    video_tensor = [i.unsqueeze(0).to(device) for i in video_tensor]  # Добавление batch dimension

    with torch.no_grad():
        outputs = model(video_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        pred = torch.argmax(outputs, dim=1).item()
        description = label_to_description[pred]

    return video_path, description, probabilities.squeeze().cpu().numpy()

# Пример использования функции run_inference_on_segments
video_path = 'testing/0000000_00000020240508132920_0006_1.mp4'
checkpoint_path = 'slowfast_epoch_5.pth'
segment_paths = split_video(video_path)
predictions = run_inference_on_segments(segment_paths, checkpoint_path)

# Функция для сохранения данных в Excel
def save_to_excel(violations, excel_path):
    # Создаем DataFrame только с полями 'file' и 'violation'
    df = pd.DataFrame(violations, columns=['file', 'violation'])
    df.to_excel(excel_path, index=False)

# Пример использования функции save_to_excel
excel_path = 'violations_report.xlsx'
save_to_excel([pred[:2] for pred in predictions], excel_path)  # Сохраняем только файл и тип нарушения


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [48]:
import os
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def split_video(video_path, segment_length=10):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    segment_paths = []
    
    for start_time in range(0, int(duration), segment_length):
        end_time = start_time + segment_length
        if end_time > duration:
            end_time = duration
        segment_path = f"{video_path.split('.')[0]}_segment_{start_time}_{end_time}.mp4"
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=segment_path)
        segment_paths.append(segment_path)
    
    cap.release()
    return segment_paths

def run_inference_on_segments(segment_paths, checkpoint_path):
    # Assuming run_inference is defined elsewhere and works with single video file paths
    predictions = []
    for segment_path in segment_paths:
        prediction = run_inference(valid_video_dir, checkpoint_path)
        print(prediction)
        predictions.append(prediction)
    return predictions

video_path = 'testing/0000000_00000020240508132920_0006_1_segment_0_5.005.mp4'
checkpoint_path = 'slowfast_epoch_5.pth'
segment_paths = split_video(video_path)
prid= run_inference_on_segments(segment_paths, checkpoint_path)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


('testing/0000000_00000020240508132920_0006_1.mp4', 'Нарушение: Работы под составом', array([9.9626714e-01, 8.6023146e-04, 2.8726631e-03], dtype=float32))


In [49]:
import os
import cv2
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def split_video(video_path, segment_length=0.1):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    segment_paths = []

    segment_length = segment_length * duration  # Преобразование относительной длины в абсолютную

    for start_time in range(0, int(duration), int(segment_length)):
        end_time = start_time + segment_length
        if end_time > duration:
            end_time = duration
        segment_path = f"{video_path.split('.')[0]}_segment_{start_time}_{end_time}.mp4"
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=segment_path)
        segment_paths.append(segment_path)
    
    cap.release()
    return segment_paths

def run_inference_on_segments(segment_paths, checkpoint_path):
    # Assuming run_inference is defined elsewhere and works with single video file paths
    predictions = []
    for segment_path in segment_paths:
        prediction = run_inference(segment_path, checkpoint_path)
        print(prediction)
        predictions.append(prediction)
    return predictions

video_path = 'testing/2_5395803543229709828_0.mp4'
checkpoint_path = 'slowfast_epoch_5.pth'
segment_paths = split_video(video_path)
predictions = run_inference_on_segments(segment_paths, checkpoint_path)

# Объединение предсказаний в последовательный вывод
for prediction in predictions:
    print(f"Файл: {prediction[0]}")
    print(f"Описание: {prediction[1]}")
    print(f"Вероятности: {prediction[2]}")
    print("------------------------")


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


('testing/2_5395803543229709828_0_segment_0_2.1021.mp4', 'Нарушение: Работы под составом', array([9.9979442e-01, 1.1063379e-04, 9.4974566e-05], dtype=float32))
('testing/2_5395803543229709828_0_segment_2_4.1021.mp4', 'Нарушение: Работы под составом', array([9.9999917e-01, 3.9661867e-07, 4.2745143e-07], dtype=float32))
('testing/2_5395803543229709828_0_segment_4_6.1021.mp4', 'Нарушение: Работы под составом', array([0.9955924 , 0.00278568, 0.00162189], dtype=float32))
('testing/2_5395803543229709828_0_segment_6_8.1021.mp4', 'Нарушение: Работы под составом', array([9.9999952e-01, 3.5211409e-07, 1.0840350e-07], dtype=float32))
('testing/2_5395803543229709828_0_segment_8_10.1021.mp4', 'Нарушение: Работы под составом', array([9.9999976e-01, 1.9966697e-07, 1.9015134e-08], dtype=float32))
('testing/2_5395803543229709828_0_segment_10_12.1021.mp4', 'Нарушение: Работы под составом', array([9.9981469e-01, 1.2809719e-04, 5.7236732e-05], dtype=float32))
('testing/2_5395803543229709828_0_segment_12_1

In [51]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from torchvision.transforms import Compose, Lambda, Normalize, Resize, CenterCrop

# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=False)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Три выхода для многоклассовой классификации
model = model.to(device)

# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

# Преобразования для видео
class VideoTransform:
    def __init__(self, side_size, mean, std, crop_size):
        self.transforms = Compose([
            Lambda(lambda x: x / 255.0),
            Resize((side_size, side_size)),
            CenterCrop(crop_size),
            Lambda(lambda x: self.normalize_video(x, mean, std))
        ])

    def __call__(self, x):
        return self.transforms(x)

    def normalize_video(self, video, mean, std):
        mean = torch.tensor(mean).view(3, 1, 1, 1)
        std = torch.tensor(std).view(3, 1, 1, 1)
        return (video - mean) / std

transform = VideoTransform(side_size, mean, std, crop_size)

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

def load_and_preprocess_video(video_path, transform, num_frames=32, sampling_rate=2):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps

    for i in range(num_frames):
        frame_id = i * sampling_rate
        if frame_id >= frame_count:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    if not frames:
        raise ValueError(f"Не удалось загрузить кадры из видео: {video_path}")

    if len(frames) < num_frames:
        # Повторение последнего кадра, если видео короче ожидаемого числа кадров
        frames += [frames[-1]] * (num_frames - len(frames))

    frames = np.stack(frames)
    frames = torch.from_numpy(frames).permute(3, 0, 1, 2)  # Преобразование в CTHW
    frames = transform(frames)  # Применение преобразований

    pack_pathway = PackPathway()
    video_tensor = pack_pathway(frames)
    return video_tensor

# Словарь для преобразования меток в текстовые описания
label_to_description = {
    0: "Нарушение: Работы под составом",
    1: "Нарушение: Передвижение на подвижном составе",
    2: "Без нарушений"
}

def run_inference(video_path: str, checkpoint_path: str):
    # Загрузка контрольной точки
    checkpoint = torch.load(checkpoint_path)

    # Восстановление весов модели
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    video_tensor = load_and_preprocess_video(video_path, transform)
    video_tensor = [i.unsqueeze(0).to(device) for i in video_tensor]  # Добавление batch dimension

    with torch.no_grad():
        outputs = model(video_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        pred = torch.argmax(outputs, dim=1).item()
        description = label_to_description[pred]

    return video_path, description, probabilities.squeeze().cpu().numpy()

def split_video(video_path, segment_length=0.1):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    segment_paths = []

    segment_length = segment_length * duration  # Преобразование относительной длины в абсолютную

    for start_time in range(0, int(duration), int(segment_length)):
        end_time = start_time + segment_length
        if end_time > duration:
            end_time = duration
        segment_path = f"{video_path.split('.')[0]}_segment_{start_time}_{end_time}.mp4"
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=segment_path)
        segment_paths.append((segment_path, start_time, end_time))
    
    cap.release()
    return segment_paths

def run_inference_on_segments(segment_paths, checkpoint_path):
    predictions = []
    for segment_path, start_time, end_time in segment_paths:
        video_path, description, probabilities = run_inference(segment_path, checkpoint_path)
        predictions.append((video_path, description, start_time, end_time))
    return predictions

# Пример использования функции split_video и run_inference_on_segments
video_path = 'testing/2_5395803543229709828_0.mp4'
checkpoint_path = 'slowfast_epoch_5.pth'
segment_paths = split_video(video_path)
predictions = run_inference_on_segments(segment_paths, checkpoint_path)

# Объединение предсказаний и сохранение в Excel
def save_predictions_to_excel(predictions, excel_path):
    data = []
    for prediction in predictions:
        file_name = os.path.basename(prediction[0])
        description = prediction[1]
        time_interval = f"{prediction[2]} - {prediction[3]}"
        data.append([file_name, description, time_interval])

    df = pd.DataFrame(data, columns=['Файл', 'Нарушение', 'Интервал'])
    df.to_excel(excel_path, index=False)

# Пример сохранения в Excel
excel_path = 'violations_report.xlsx'
save_predictions_to_excel(predictions, excel_path)


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [53]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from torchvision.transforms import Compose, Lambda, Normalize, Resize, CenterCrop

# Установка устройства для вычислений (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка предобученной модели
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=False)
model.blocks[-1].proj = torch.nn.Linear(model.blocks[-1].proj.in_features, 3)  # Три выхода для многоклассовой классификации
model = model.to(device)

# Параметры для обработки видео
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4

# Преобразования для видео
class VideoTransform:
    def __init__(self, side_size, mean, std, crop_size):
        self.transforms = Compose([
            Lambda(lambda x: x / 255.0),
            Resize((side_size, side_size)),
            CenterCrop(crop_size),
            Lambda(lambda x: self.normalize_video(x, mean, std))
        ])

    def __call__(self, x):
        return self.transforms(x)

    def normalize_video(self, video, mean, std):
        mean = torch.tensor(mean).view(3, 1, 1, 1)
        std = torch.tensor(std).view(3, 1, 1, 1)
        return (video - mean) / std

transform = VideoTransform(side_size, mean, std, crop_size)

class PackPathway(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

def load_and_preprocess_video(video_path, transform, num_frames=32, sampling_rate=2):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps

    for i in range(num_frames):
        frame_id = i * sampling_rate
        if frame_id >= frame_count:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    if not frames:
        raise ValueError(f"Не удалось загрузить кадры из видео: {video_path}")

    if len(frames) < num_frames:
        # Повторение последнего кадра, если видео короче ожидаемого числа кадров
        frames += [frames[-1]] * (num_frames - len(frames))

    frames = np.stack(frames)
    frames = torch.from_numpy(frames).permute(3, 0, 1, 2)  # Преобразование в CTHW
    frames = transform(frames)  # Применение преобразований

    pack_pathway = PackPathway()
    video_tensor = pack_pathway(frames)
    return video_tensor

# Словарь для преобразования меток в текстовые описания
label_to_description = {
    0: "Нарушение: Работы под составом",
    1: "Нарушение: Передвижение на подвижном составе",
    2: "Без нарушений"
}

def run_inference(video_path: str, checkpoint_path: str):
    # Загрузка контрольной точки
    checkpoint = torch.load(checkpoint_path)

    # Восстановление весов модели
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    video_tensor = load_and_preprocess_video(video_path, transform)
    video_tensor = [i.unsqueeze(0).to(device) for i in video_tensor]  # Добавление batch dimension

    with torch.no_grad():
        outputs = model(video_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        pred = torch.argmax(outputs, dim=1).item()
        description = label_to_description[pred]

    return video_path, description, probabilities.squeeze().cpu().numpy()

def split_video(video_path, segment_length=0.1):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = total_frames / fps
    segment_paths = []

    segment_length = segment_length * duration  # Преобразование относительной длины в абсолютную

    for start_time in range(0, int(duration), int(segment_length)):
        end_time = start_time + segment_length
        if end_time > duration:
            end_time = duration
        segment_path = f"{video_path.split('.')[0]}_segment_{start_time}_{end_time}.mp4"
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=segment_path)
        segment_paths.append((segment_path, start_time, end_time))
    
    cap.release()
    return segment_paths

def run_inference_on_segments(segment_paths, checkpoint_path):
    predictions = []
    for segment_path, start_time, end_time in segment_paths:
        video_path, description, probabilities = run_inference(segment_path, checkpoint_path)
        if predictions and predictions[-1][1] == description:
            # Обновление времени окончания предыдущего предсказания
            predictions[-1][3] = end_time
        else:
            predictions.append([video_path, description, start_time, end_time])
    return predictions

# Пример использования функции split_video и run_inference_on_segments
video_path = 'testing/2_5395803543229709828_0.mp4'
checkpoint_path = 'slowfast_epoch_5.pth'
segment_paths = split_video(video_path)
predictions = run_inference_on_segments(segment_paths, checkpoint_path)

# Объединение предсказаний и сохранение в Excel
def save_predictions_to_excel(predictions, excel_path):
    data = []
    for prediction in predictions:
        file_name = os.path.basename(prediction[0])
        description = prediction[1]
        time_interval = f"{prediction[2]} - {prediction[3]}"
        data.append([file_name, description, time_interval])

    df = pd.DataFrame(data, columns=['Файл', 'Нарушение', 'Интервал'])
    df.to_excel(excel_path, index=False)

# Пример сохранения в Excel
excel_path = 'violations_report.xlsx'
save_predictions_to_excel(predictions, excel_path)


Using cache found in /Users/bogdanroshchupkin/.cache/torch/hub/facebookresearch_pytorchvideo_main


Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


/Users/bogdanroshchupkin/anaconda3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
